# Creating data splits

In [57]:
import os
import pandas as pd
import random
import math
import numpy as np


# Set seed for random
random.seed(10)
np.random.seed(42)


##########################################################################
##########################################################################


oulu_casia_mongo_patient_ids = [7, 8, 9, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60,
                                61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79]

e_eval = ['Neutral', 'Happy', 'Sad', 'Surprise', 'Fear', 'Disgust', 'Anger', 'Contempt', 'None']
e_dict = {0:'Neutral', 1:'Happy', 2:'Sad', 3:'Surprise', 4:'Fear', 5:'Disgust', 6:'Anger', 7:'Contempt', 8:'None'}
# e_eval_8 = ['Neutral', 'Happy', 'Sad', 'Surprise', 'Fear', 'Disgust', 'Anger', 'Contempt']
# e_dict_8 = {0:'Neutral', 1:'Happy', 2:'Sad', 3:'Surprise', 4:'Fear', 5:'Disgust', 6:'Anger', 7:'Contempt'}
# e_dict_8_inv = {'Neutral':0, 'Happy':1, 'Sad':2, 'Surprise':3, 'Fear':4, 'Disgust':5, 'Anger':6, 'Contempt':7}
# e_eval_8_wrongorder = ['Neutral', 'Happy', 'Anger', 'Sad', 'Fear', 'Surprise', 'Disgust', 'Contempt']
# e_dict_8_wrongorder = {0: 'Neutral', 1:'Happy', 2:'Anger', 3:'Sad', 4:'Fear', 5:'Surprise', 6:'Disgust', 7:'Contempt'}
# e_eval_8_alphabetic = ['Anger','Contempt','Disgust','Fear','Happy','Neutral','Sad','Surprise']
# e_dict_8_alphabetic = {0:'Anger',1:'Contempt',2:'Disgust',3:'Fear',4:'Happy',5:'Neutral',6:'Sad',7:'Surprise'}
e_dict_8_lower2upper = {'neutral':"Neutral", "happiness":"Happy", "sadness":"Sad", "surprise":"Surprise", "fear": "Fear", "disgust": "Disgust", "anger": "Anger", "contempt": "Contempt"}
e_dict_8_oulucasia2id = {'neutral':0., "happiness":1., "sadness":2., "surprise":3., "fear": 4., "disgust": 5., "anger": 6., "contempt": 7.}

# Define valence-arousal values for emotions
# anchors inspired by https://arxiv.org/pdf/2311.14816.pdf
valence_arousal = {
    "neutral": (0., 0.),
    "anger": (-0.51, 0.59),
    "disgust": (-0.60, 0.35),
    "fear": (-0.64, 0.6),
    "happiness": (0.81, 0.51),
    "sadness": (-0.63, -0.27),
    "surprise": (0.4, 0.67)
}

va_recalculate2square_range = False


##########################################################################
##########################################################################


def magnitude(vector):
    return math.sqrt(sum(pow(element, 2) for element in vector))


def circumplex2square(a):
    a = np.array(a)
    a_magn = magnitude(a)

    if a_magn > 1:
        print("Not in range")
        return

    if a[0] == 0 or a[1] == 0:
        return a

    a1_sign = -1 if a[0] < 0 else 1
    a2_sign = -1 if a[1] < 0 else 1
    if abs(a[0] / a[1]) > 1:
        point_on_square = (1., abs(1/a[0]*a[1]))
    else:
        point_on_square = (abs(1/a[1]*a[0]), 1.)
    point_on_square = np.array(
        [a1_sign*point_on_square[0], a2_sign*point_on_square[1]])

    b = a_magn * point_on_square

    return b

# Recalculate if range [-1, 1] is used for oulucasiadb
if va_recalculate2square_range:
    for k, v in valence_arousal.items():
        valence_arousal[k] = circumplex2square(v)


def label_emotion_images(imgs, emotion):
    cat_emotion = []
    spat_emotion = []

    ##############
    # STRATEGY 1 #
    ##############
    # for i, im in enumerate(imgs):
    #     # if first quarter -> neutral, else default emotion
    #     cat_emotion.append("neutral" if i < (len(imgs) / 4) else emotion.lower())

    # # spatial emotions
    # spat_emotion = np.linspace(valence_arousal["neutral"], valence_arousal[emotion.lower()], len(imgs))

    ##############
    # STRATEGY 2 #
    ##############
    for i, im in enumerate(imgs):
        # if first quarter -> neutral, else default emotion
        cat_emotion.append("neutral" if i < (
            len(imgs) / 4) else emotion.lower())

        if i < (len(imgs) / 4):
            spat_emotion.append(valence_arousal["neutral"])
        elif i < (len(imgs) / 4 * 3):
            spat_emotion.append((np.array(
                valence_arousal["neutral"]) + np.array(valence_arousal[emotion.lower()])) / 2)
        else:
            spat_emotion.append(valence_arousal[emotion.lower()])

    return cat_emotion, spat_emotion


def get_files_oulucasia(path, spectra, add_annotations=True):
    @staticmethod
    def get_annotations_oulucasia(df):
        grouped = df.groupby(
            ['oulucasia-patient', 'oulucasia-emotion'])['filename'].apply(list)

        patient_emotion_list = [(filenames, emotion) for (
            patient, emotion), filenames in grouped.items()]
        labels = [label_emotion_images(imgs, emotion)
                  for imgs, emotion in patient_emotion_list]
        expression_list = [l[0] for l in labels]
        valence_list = [l[1] for l in labels]
        arousal_list = [l[1] for l in labels]
 
        flat_expression_list = []
        for row in expression_list:
            flat_expression_list.extend(row)
        flat_valence_list = []
        for row in valence_list:
            flat_valence_list.extend([r[0] for r in row])
        flat_arousal_list = []
        for row in arousal_list:
            flat_arousal_list.extend([r[1] for r in row])

        return flat_expression_list, flat_valence_list, flat_arousal_list

    filepath_list = []
    filename_list = []
    adjusted_unique_filename_list = []
    oulucasia_patient_list = []
    oulucasia_emotion_list = []
    race_list = []
    filetype_list = []

    for i, p in enumerate(sorted(os.listdir(path))):
        for j, e in enumerate(sorted(os.listdir(os.path.join(path, p)))):
            for k, f in enumerate(sorted(os.listdir(os.path.join(path, p, e)))):
                fp = os.path.join(path, p, e, f)
                race = "mongo" if int(
                    p[1:]) in oulu_casia_mongo_patient_ids else "cauca"
                adjusted_unique_filename = "-".join(["oulucasia", p, e, f])

                filepath_list.append(str(fp))
                filename_list.append(f)
                adjusted_unique_filename_list.append(adjusted_unique_filename)
                oulucasia_patient_list.append(p)
                oulucasia_emotion_list.append(e)
                race_list.append(race)
                filetype_list.append(f.split('.')[-1])

    df = pd.DataFrame({
        "db": "oulucasia",
        "spectra": spectra,
        "filepath": filepath_list,
        "filename": filename_list,
        "adjusted_unique_filename": adjusted_unique_filename_list,
        "oulucasia-patient": oulucasia_patient_list,
        "oulucasia-emotion": oulucasia_emotion_list,
        "filetype": filetype_list,
        "race": race_list,
    })

    if add_annotations:
        expression_list, valence_list, arousal_list = get_annotations_oulucasia(
            df)
        df["expression"] = expression_list
        df["valence"] = valence_list
        df["arousal"] = arousal_list
        df["expression_name"] = df.expression.apply(lambda x: e_dict_8_lower2upper[x])
        df["expression"] = df.expression.apply(lambda x: e_dict_8_oulucasia2id[x])
        

    return df


def get_files_casia(path, spectra):
    filenames = os.listdir(path)
    filepaths = [str(os.path.join(path, f)) for f in filenames]
    adjusted_unique_filename_list = ["-".join(["casia", f]) for f in filenames]
    session_list = [int(f.split('_')[0][1:]) for f in filenames]
    patient_list = [int(f.split('_')[2]) for f in filenames]
    emotion_list = [int(f.split('_')[3].split('.')[0].split('-')[0])
                    for f in filenames]
    filetype_list = [f.split('.')[-1] for f in filenames]

    df = pd.DataFrame({
        "db": "casia",
        "spectra": spectra,
        "filepath": filepaths,
        "filename": filenames,
        "adjusted_unique_filename": adjusted_unique_filename_list,
        "casia-session": session_list,
        "casia-patient": patient_list,
        "casia-img_id": emotion_list,
        "filetype": filetype_list,
        "race": "mongo",
    })

    return df


def get_files_buaa(path, labels_fp=None):
    @staticmethod
    def get_buaa_filetype(i, f):
        if int(f.split('.')[0])>30:
            return "low_light"
        
        type_ = ''
        if i < 2:
            type_ = "neutral"
        elif i < 18:
            type_ = "tilt"
        elif i < 26:
            type_ = "affection"
        else:
            type_ = "low_light"

        return type_

    filepath_list = []
    filename_list = []
    adjusted_unique_filename_list = []
    buaa_patient_list = []
    filetype_list = []
    buaa_image_type = []
    spectra_list = []

    for p in os.listdir(path):
        if not os.path.isdir(os.path.join(path, p)):
            continue
        dir_content_images = [f for f in os.listdir(os.path.join(path, p)) if f.split('.')[-1] == 'bmp']
        for i, f in enumerate(sorted(dir_content_images, key=lambda x: int(x.split(".")[0]))):
            filename_list.append(f)
            filepath_list.append(str(os.path.join(path, p, f)))
            adjusted_unique_filename_list.append("-".join(["buaa", p, f]))
            buaa_patient_list.append(str(p))
            filetype_list.append(f.split(".")[-1])
            buaa_image_type.append(get_buaa_filetype(i, f))
            spectra_list.append("vis" if i % 2 == 0 else "nir")

    df = pd.DataFrame({
        "db": "buaa",
        "spectra": spectra_list,
        "filepath": filepath_list,
        "filename": filename_list,
        "adjusted_unique_filename": adjusted_unique_filename_list,
        "buaa-patient": buaa_patient_list,
        "buaa-image_type": buaa_image_type,
        "filetype": filetype_list,
        "race": "mongo",
    })
    
    if labels_fp:
        df_annotations = pd.read_csv(labels_fp)
        df_annotations["adjusted_unique_filename"] = df_annotations.filename.apply(lambda x: "-".join(x.split("_")))
        df = df.merge(df_annotations[[
                      'adjusted_unique_filename', 'expression', 'valence', 'arousal']], on='adjusted_unique_filename', how='left')
        df["expression"].fillna(0., inplace=True)
        df["valence"].fillna(0., inplace=True)
        df["arousal"].fillna(0., inplace=True)
        df["expression_name"] = df["expression"].apply(lambda x: e_dict[int(x)])
        
        df.loc[0::2, ["expression","expression_name","valence","arousal"]] = df.loc[1::2,["expression","expression_name","valence","arousal"]].values
        
    return df


def get_files_customdb(path, labels_fp=None):
    filenames = os.listdir(path)
    filepaths = [str(os.path.join(path, f)) for f in filenames]
    adjusted_unique_filename_list = [
        "-".join(["customdb", f]) for f in filenames]
    patient_list = [int(f.split('-')[0]) for f in filenames]
    filetype_list = [f.split('.')[-1] for f in filenames]

    df = pd.DataFrame({
        "db": "customdb",
        "spectra": "nir",
        "filepath": filepaths,
        "filename": filenames,
        "adjusted_unique_filename": adjusted_unique_filename_list,
        "customdb-patient": patient_list,
        "filetype": filetype_list,
        "race": "cauca",
    })

    if labels_fp:
        df_annotations = pd.read_csv(labels_fp)
        df = df.merge(df_annotations[[
                      'filename', 'expression', 'valence', 'arousal']], on='filename', how='left')
        df["expression_name"] = df["expression"].apply(lambda x: e_dict[int(x)])

    return df

def get_files_affectnet(path, num_samples, labels_fp):
    df = pd.read_csv(labels_fp)
    df = df[df["exists_in_dir"]==True]
    df = df.rename(columns={"FileType": "filetype"})
    df['filename'] = df['Unnamed: 0'].apply(lambda x: str(x) + ".jpg")
    df = df[["expression", "valence", "arousal", "filetype", "filename"]]
    df["expression_name"] = df["expression"].apply(lambda x: e_dict[int(x)])
    
    # sample data
    df = df.sample(n=num_samples)
    
    # get filetypes to lower
    df["filetype"] = df["filetype"].apply(lambda x: x.lower())
    
    # add data
    df["filepath"] = [str(os.path.join(path, f)) for f in df.filename]
    df["adjusted_unique_filename"] = [
        "-".join(["affectnet", f]) for f in df.filename]
    df["db"] = "affectnet"
    df["spectra"] = "vis"
    df["race"] = "cauca"

    return df

def get_overview(df):
    # Assuming 'df' is your DataFrame and it has columns 'db', 'spectra', and 'race'
    nir_mongo_counts = df[(df['spectra'] == 'nir') & (df['race'] == 'mongo')].groupby('db').size()
    nir_cauca_counts = df[(df['spectra'] == 'nir') & (df['race'] == 'cauca')].groupby('db').size()
    vis_mongo_counts = df[(df['spectra'] == 'vis') & (df['race'] == 'mongo')].groupby('db').size()
    vis_cauca_counts = df[(df['spectra'] == 'vis') & (df['race'] == 'cauca')].groupby('db').size()
    nir_counts = df[(df['spectra'] == 'nir')].groupby('db').size()
    vis_counts = df[(df['spectra'] == 'vis')].groupby('db').size()

    result = pd.DataFrame({
        '#NIR': nir_counts,
        '#VIS': vis_counts
    })

    result['#NIR mongo'] = nir_mongo_counts
    result['#NIR cauca'] = nir_cauca_counts
    result['#VIS mongo'] = vis_mongo_counts
    result['#VIS cauca'] = vis_cauca_counts

    # Fill NaN values with 0
    result = result.fillna(0)

    # Convert float counts to int
    for col in result.columns:
        result[col] = result[col].astype(int)

    # display(result)
    # Add a total row
    result.loc['Total'] = result.sum()

    # Convert float counts to int
    for col in result.columns:
        result[col] = result[col].astype(int)
        
    return result

Load all annotations of all images except the AffectNet as a whole - only small part

In [58]:
df_oulucasia_vl = get_files_oulucasia('data/B_OriginalImg/VL/Strong/', "vis")
df_oulucasia_ni = get_files_oulucasia('data/B_OriginalImg/NI/Strong/', "nir")
df_casia_vl = get_files_casia('data/CASIA_NISVIR/NIR-VIS/VIS/', "vis")
df_casia_ni = get_files_casia('data/CASIA_NISVIR/NIR-VIS/NIR/',"nir")
df_buaa_vl_ni = get_files_buaa('data/BUAA/BUAAVISNIR/', labels_fp="Custom_DB/buaa_w_neutral-annotations.csv")
df_customdb_ni = get_files_customdb('Custom_DB/custom_nir/', labels_fp="Custom_DB/custom_nir-annotations.csv")
df_affectnet_vl = get_files_affectnet('data/AffectNet-8Labels/train_set/images/', 2086, labels_fp="data/AffectNet-8Labels/train_custom.csv")

# Concatenate all
df = pd.concat([df_oulucasia_vl, df_oulucasia_ni, df_casia_vl, df_casia_ni, df_buaa_vl_ni, df_customdb_ni, df_affectnet_vl], ignore_index=True)

# Drop all nonimage files
df = df[df["filetype"].isin(['jpeg', 'jpg', 'bmp', 'png'])]

# Change format of images to 'jpg' - due to its compact size
df['new_filetype'] = 'jpg'
df['adjusted_unique_filename'] = df['adjusted_unique_filename'].apply(lambda x: x.split('.')[0] + '.jpg')

Some records will be dropped, because many images in databases are either very similar, or inappropriate for model training

In [59]:
# In OuluCasia - get 5 samples per patient per emotion
df_cyclegan_all = df.groupby(['spectra', 'oulucasia-patient', 'oulucasia-emotion'], as_index=False).apply(lambda x: x.sample(5)).reset_index(drop=True)
df_cyclegan_all = pd.concat([df[df["db"]!="oulucasia"], df_cyclegan_all], ignore_index=True)

# In CASIA - get 2 samples per patient/session
casia_grouped = df.groupby(['spectra', 'casia-patient', 'casia-session'], as_index=False).apply(lambda x: x.sample(1)).reset_index(drop=True)
df_cyclegan_all = pd.concat([df_cyclegan_all[df_cyclegan_all["db"]!="casia"], casia_grouped], ignore_index=True)

# In BUAA - drop low-light images and images from VIS spectra
df_cyclegan_all = df_cyclegan_all[~((df_cyclegan_all["buaa-image_type"]=="low_light")|((df_cyclegan_all["db"]=="buaa")&(df_cyclegan_all["spectra"]=="vis")))]

display(df_cyclegan_all)

,db,spectra,filepath,filename,adjusted_unique_filename,oulucasia-patient,oulucasia-emotion,filetype,race,expression,valence,arousal,expression_name,casia-session,casia-patient,casia-img_id,buaa-patient,buaa-image_type,customdb-patient,new_filetype
1,buaa,nir,data/BUAA/BUAAVISNIR/19/2.bmp,2.bmp,buaa-19-2.jpg,NaN,NaN,bmp,mongo,0.0,0.0,0.0,Neutral,NaN,NaN,NaN,19,neutral,NaN,jpg
3,buaa,nir,data/BUAA/BUAAVISNIR/19/4.bmp,4.bmp,buaa-19-4.jpg,NaN,NaN,bmp,mongo,0.0,0.0,0.0,Neutral,NaN,NaN,NaN,19,tilt,NaN,jpg
5,buaa,nir,data/BUAA/BUAAVISNIR/19/6.bmp,6.bmp,buaa-19-6.jpg,NaN,NaN,bmp,mongo,0.0,0.0,0.0,Neutral,NaN,NaN,NaN,19,tilt,NaN,jpg
7,buaa,nir,data/BUAA/BUAAVISNIR/19/8.bmp,8.bmp,buaa-19-8.jpg,NaN,NaN,bmp,mongo,0.0,0.0,0.0,Neutral,NaN,NaN,NaN,19,tilt,NaN,jpg
9,buaa,nir,data/BUAA/BUAAVISNIR/19/10.bmp,10.bmp,buaa-19-10.jpg,NaN,NaN,bmp,mongo,0.0,0.0,0.0,Neutral,NaN,NaN,NaN,19,tilt,NaN,jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14461,casia,vis,data/CASIA_NISVIR/NIR-VIS/VIS/s4_VIS_30785_001...,s4_VIS_30785_001.jpg,casia-s4_VIS_30785_001.jpg,NaN,NaN,jpg,mongo,NaN,NaN,NaN,NaN,4.0,30785.0,1.0,NaN,NaN,NaN,jpg
14462,casia,vis,data/CASIA_NISVIR/NIR-VIS/VIS/s4_VIS_30786_002...,s4_VIS_30786_002.jpg,casia-s4_VIS_30786_002.jpg,NaN,NaN,jpg,mongo,NaN,NaN,NaN,NaN,4.0,30786.0,2.0,NaN,NaN,NaN,jpg
14463,casia,vis,data/CASIA_NISVIR/NIR-VIS/VIS/s4_VIS_30787_003...,s4_VIS_30787_003.jpg,casia-s4_VIS_30787_003.jpg,NaN,NaN,jpg,mongo,NaN,NaN,NaN,NaN,4.0,30787.0,3.0,NaN,NaN,NaN,jpg
14464,casia,vis,data/CASIA_NISVIR/NIR-VIS/VIS/s4_VIS_30836_003...,s4_VIS_30836_003.jpg,casia-s4_VIS_30836_003.jpg,NaN,NaN,jpg,mongo,NaN,NaN,NaN,NaN,4.0,30836.0,3.0,NaN,NaN,NaN,jpg


Summarize the data

In [60]:
display(get_overview(df))

,#NIR,#VIS,#NIR mongo,#NIR cauca,#VIS mongo,#VIS cauca
db,,,,,,
affectnet,0,2086,0,0,0,2086
buaa,2976,2976,2976,0,2976,0
casia,12487,5093,12487,0,5093,0
customdb,159,0,0,159,0,0
oulucasia,10379,10379,4338,6041,4206,6173
Total,26001,20534,19801,6200,12275,8259


display examples of those

In [61]:
display(df_cyclegan_all[df_cyclegan_all["spectra"]=="vis"])
display(df_cyclegan_all[df_cyclegan_all["spectra"]=="nir"])
display(df_cyclegan_all[(df_cyclegan_all["spectra"]=="vis") & (df_cyclegan_all["db"]=="affectnet")])
display(df_cyclegan_all[(df_cyclegan_all["spectra"]=="vis") & (df_cyclegan_all["db"]=="oulucasia")])
display(df_cyclegan_all[(df_cyclegan_all["spectra"]=="nir") & (df_cyclegan_all["db"]=="oulucasia")])
display(df_cyclegan_all[(df_cyclegan_all["spectra"]=="vis") & (df_cyclegan_all["db"]=="casia")])
display(df_cyclegan_all[(df_cyclegan_all["spectra"]=="nir") & (df_cyclegan_all["db"]=="casia")])
display(df_cyclegan_all[(df_cyclegan_all["spectra"]=="vis") & (df_cyclegan_all["db"]=="buaa")])
display(df_cyclegan_all[(df_cyclegan_all["spectra"]=="nir") & (df_cyclegan_all["db"]=="buaa")])
display(df_cyclegan_all[(df_cyclegan_all["spectra"]=="nir") & (df_cyclegan_all["db"]=="customdb")])

,db,spectra,filepath,filename,adjusted_unique_filename,oulucasia-patient,oulucasia-emotion,filetype,race,expression,valence,arousal,expression_name,casia-session,casia-patient,casia-img_id,buaa-patient,buaa-image_type,customdb-patient,new_filetype
6111,affectnet,vis,data/AffectNet-8Labels/train_set/images/216931...,216931.jpg,affectnet-216931.jpg,NaN,NaN,jpg,cauca,1.0,0.512969,0.058072,Happy,NaN,NaN,NaN,NaN,NaN,NaN,jpg
6112,affectnet,vis,data/AffectNet-8Labels/train_set/images/310233...,310233.jpg,affectnet-310233.jpg,NaN,NaN,png,cauca,0.0,0.000000,-0.007937,Neutral,NaN,NaN,NaN,NaN,NaN,NaN,jpg
6113,affectnet,vis,data/AffectNet-8Labels/train_set/images/197479...,197479.jpg,affectnet-197479.jpg,NaN,NaN,jpg,cauca,1.0,0.595238,0.015873,Happy,NaN,NaN,NaN,NaN,NaN,NaN,jpg
6114,affectnet,vis,data/AffectNet-8Labels/train_set/images/262013...,262013.jpg,affectnet-262013.jpg,NaN,NaN,jpg,cauca,1.0,0.900116,0.135501,Happy,NaN,NaN,NaN,NaN,NaN,NaN,jpg
6115,affectnet,vis,data/AffectNet-8Labels/train_set/images/402859...,402859.jpg,affectnet-402859.jpg,NaN,NaN,jpg,cauca,1.0,0.832365,0.106465,Happy,NaN,NaN,NaN,NaN,NaN,NaN,jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14461,casia,vis,data/CASIA_NISVIR/NIR-VIS/VIS/s4_VIS_30785_001...,s4_VIS_30785_001.jpg,casia-s4_VIS_30785_001.jpg,NaN,NaN,jpg,mongo,NaN,NaN,NaN,NaN,4.0,30785.0,1.0,NaN,NaN,NaN,jpg
14462,casia,vis,data/CASIA_NISVIR/NIR-VIS/VIS/s4_VIS_30786_002...,s4_VIS_30786_002.jpg,casia-s4_VIS_30786_002.jpg,NaN,NaN,jpg,mongo,NaN,NaN,NaN,NaN,4.0,30786.0,2.0,NaN,NaN,NaN,jpg
14463,casia,vis,data/CASIA_NISVIR/NIR-VIS/VIS/s4_VIS_30787_003...,s4_VIS_30787_003.jpg,casia-s4_VIS_30787_003.jpg,NaN,NaN,jpg,mongo,NaN,NaN,NaN,NaN,4.0,30787.0,3.0,NaN,NaN,NaN,jpg
14464,casia,vis,data/CASIA_NISVIR/NIR-VIS/VIS/s4_VIS_30836_003...,s4_VIS_30836_003.jpg,casia-s4_VIS_30836_003.jpg,NaN,NaN,jpg,mongo,NaN,NaN,NaN,NaN,4.0,30836.0,3.0,NaN,NaN,NaN,jpg


,db,spectra,filepath,filename,adjusted_unique_filename,oulucasia-patient,oulucasia-emotion,filetype,race,expression,valence,arousal,expression_name,casia-session,casia-patient,casia-img_id,buaa-patient,buaa-image_type,customdb-patient,new_filetype
1,buaa,nir,data/BUAA/BUAAVISNIR/19/2.bmp,2.bmp,buaa-19-2.jpg,NaN,NaN,bmp,mongo,0.0,0.0,0.0,Neutral,NaN,NaN,NaN,19,neutral,NaN,jpg
3,buaa,nir,data/BUAA/BUAAVISNIR/19/4.bmp,4.bmp,buaa-19-4.jpg,NaN,NaN,bmp,mongo,0.0,0.0,0.0,Neutral,NaN,NaN,NaN,19,tilt,NaN,jpg
5,buaa,nir,data/BUAA/BUAAVISNIR/19/6.bmp,6.bmp,buaa-19-6.jpg,NaN,NaN,bmp,mongo,0.0,0.0,0.0,Neutral,NaN,NaN,NaN,19,tilt,NaN,jpg
7,buaa,nir,data/BUAA/BUAAVISNIR/19/8.bmp,8.bmp,buaa-19-8.jpg,NaN,NaN,bmp,mongo,0.0,0.0,0.0,Neutral,NaN,NaN,NaN,19,tilt,NaN,jpg
9,buaa,nir,data/BUAA/BUAAVISNIR/19/10.bmp,10.bmp,buaa-19-10.jpg,NaN,NaN,bmp,mongo,0.0,0.0,0.0,Neutral,NaN,NaN,NaN,19,tilt,NaN,jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13727,casia,nir,data/CASIA_NISVIR/NIR-VIS/NIR/s4_NIR_30785_008...,s4_NIR_30785_008.bmp,casia-s4_NIR_30785_008.jpg,NaN,NaN,bmp,mongo,NaN,NaN,NaN,NaN,4.0,30785.0,8.0,NaN,NaN,NaN,jpg
13728,casia,nir,data/CASIA_NISVIR/NIR-VIS/NIR/s4_NIR_30786_001...,s4_NIR_30786_001.bmp,casia-s4_NIR_30786_001.jpg,NaN,NaN,bmp,mongo,NaN,NaN,NaN,NaN,4.0,30786.0,1.0,NaN,NaN,NaN,jpg
13729,casia,nir,data/CASIA_NISVIR/NIR-VIS/NIR/s4_NIR_30787_016...,s4_NIR_30787_016.bmp,casia-s4_NIR_30787_016.jpg,NaN,NaN,bmp,mongo,NaN,NaN,NaN,NaN,4.0,30787.0,16.0,NaN,NaN,NaN,jpg
13730,casia,nir,data/CASIA_NISVIR/NIR-VIS/NIR/s4_NIR_30836_001...,s4_NIR_30836_001.bmp,casia-s4_NIR_30836_001.jpg,NaN,NaN,bmp,mongo,NaN,NaN,NaN,NaN,4.0,30836.0,1.0,NaN,NaN,NaN,jpg


,db,spectra,filepath,filename,adjusted_unique_filename,oulucasia-patient,oulucasia-emotion,filetype,race,expression,valence,arousal,expression_name,casia-session,casia-patient,casia-img_id,buaa-patient,buaa-image_type,customdb-patient,new_filetype
6111,affectnet,vis,data/AffectNet-8Labels/train_set/images/216931...,216931.jpg,affectnet-216931.jpg,NaN,NaN,jpg,cauca,1.0,0.512969,0.058072,Happy,NaN,NaN,NaN,NaN,NaN,NaN,jpg
6112,affectnet,vis,data/AffectNet-8Labels/train_set/images/310233...,310233.jpg,affectnet-310233.jpg,NaN,NaN,png,cauca,0.0,0.000000,-0.007937,Neutral,NaN,NaN,NaN,NaN,NaN,NaN,jpg
6113,affectnet,vis,data/AffectNet-8Labels/train_set/images/197479...,197479.jpg,affectnet-197479.jpg,NaN,NaN,jpg,cauca,1.0,0.595238,0.015873,Happy,NaN,NaN,NaN,NaN,NaN,NaN,jpg
6114,affectnet,vis,data/AffectNet-8Labels/train_set/images/262013...,262013.jpg,affectnet-262013.jpg,NaN,NaN,jpg,cauca,1.0,0.900116,0.135501,Happy,NaN,NaN,NaN,NaN,NaN,NaN,jpg
6115,affectnet,vis,data/AffectNet-8Labels/train_set/images/402859...,402859.jpg,affectnet-402859.jpg,NaN,NaN,jpg,cauca,1.0,0.832365,0.106465,Happy,NaN,NaN,NaN,NaN,NaN,NaN,jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8192,affectnet,vis,data/AffectNet-8Labels/train_set/images/287433...,287433.jpg,affectnet-287433.jpg,NaN,NaN,jpg,cauca,1.0,0.531746,0.007937,Happy,NaN,NaN,NaN,NaN,NaN,NaN,jpg
8193,affectnet,vis,data/AffectNet-8Labels/train_set/images/68206.jpg,68206.jpg,affectnet-68206.jpg,NaN,NaN,jpg,cauca,1.0,0.848083,-0.089549,Happy,NaN,NaN,NaN,NaN,NaN,NaN,jpg
8194,affectnet,vis,data/AffectNet-8Labels/train_set/images/122981...,122981.jpg,affectnet-122981.jpg,NaN,NaN,jpg,cauca,1.0,0.759144,0.107833,Happy,NaN,NaN,NaN,NaN,NaN,NaN,jpg
8195,affectnet,vis,data/AffectNet-8Labels/train_set/images/395128...,395128.jpg,affectnet-395128.jpg,NaN,NaN,jpg,cauca,0.0,-0.431943,0.168563,Neutral,NaN,NaN,NaN,NaN,NaN,NaN,jpg


,db,spectra,filepath,filename,adjusted_unique_filename,oulucasia-patient,oulucasia-emotion,filetype,race,expression,valence,arousal,expression_name,casia-session,casia-patient,casia-img_id,buaa-patient,buaa-image_type,customdb-patient,new_filetype
10597,oulucasia,vis,data/B_OriginalImg/VL/Strong/P001/Anger/000.jpeg,000.jpeg,oulucasia-P001-Anger-000.jpg,P001,Anger,jpeg,cauca,0.0,0.000,0.000,Neutral,NaN,NaN,NaN,NaN,NaN,NaN,jpg
10598,oulucasia,vis,data/B_OriginalImg/VL/Strong/P001/Anger/010.jpeg,010.jpeg,oulucasia-P001-Anger-010.jpg,P001,Anger,jpeg,cauca,6.0,-0.255,0.295,Anger,NaN,NaN,NaN,NaN,NaN,NaN,jpg
10599,oulucasia,vis,data/B_OriginalImg/VL/Strong/P001/Anger/014.jpeg,014.jpeg,oulucasia-P001-Anger-014.jpg,P001,Anger,jpeg,cauca,6.0,-0.510,0.590,Anger,NaN,NaN,NaN,NaN,NaN,NaN,jpg
10600,oulucasia,vis,data/B_OriginalImg/VL/Strong/P001/Anger/016.jpeg,016.jpeg,oulucasia-P001-Anger-016.jpg,P001,Anger,jpeg,cauca,6.0,-0.510,0.590,Anger,NaN,NaN,NaN,NaN,NaN,NaN,jpg
10601,oulucasia,vis,data/B_OriginalImg/VL/Strong/P001/Anger/009.jpeg,009.jpeg,oulucasia-P001-Anger-009.jpg,P001,Anger,jpeg,cauca,6.0,-0.255,0.295,Anger,NaN,NaN,NaN,NaN,NaN,NaN,jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12992,oulucasia,vis,data/B_OriginalImg/VL/Strong/P080/Surprise/028...,028.jpeg,oulucasia-P080-Surprise-028.jpg,P080,Surprise,jpeg,cauca,3.0,0.400,0.670,Surprise,NaN,NaN,NaN,NaN,NaN,NaN,jpg
12993,oulucasia,vis,data/B_OriginalImg/VL/Strong/P080/Surprise/015...,015.jpeg,oulucasia-P080-Surprise-015.jpg,P080,Surprise,jpeg,cauca,3.0,0.200,0.335,Surprise,NaN,NaN,NaN,NaN,NaN,NaN,jpg
12994,oulucasia,vis,data/B_OriginalImg/VL/Strong/P080/Surprise/026...,026.jpeg,oulucasia-P080-Surprise-026.jpg,P080,Surprise,jpeg,cauca,3.0,0.400,0.670,Surprise,NaN,NaN,NaN,NaN,NaN,NaN,jpg
12995,oulucasia,vis,data/B_OriginalImg/VL/Strong/P080/Surprise/002...,002.jpeg,oulucasia-P080-Surprise-002.jpg,P080,Surprise,jpeg,cauca,0.0,0.000,0.000,Neutral,NaN,NaN,NaN,NaN,NaN,NaN,jpg


,db,spectra,filepath,filename,adjusted_unique_filename,oulucasia-patient,oulucasia-emotion,filetype,race,expression,valence,arousal,expression_name,casia-session,casia-patient,casia-img_id,buaa-patient,buaa-image_type,customdb-patient,new_filetype
8197,oulucasia,nir,data/B_OriginalImg/NI/Strong/P001/Anger/009.jpeg,009.jpeg,oulucasia-P001-Anger-009.jpg,P001,Anger,jpeg,cauca,6.0,-0.255,0.295,Anger,NaN,NaN,NaN,NaN,NaN,NaN,jpg
8198,oulucasia,nir,data/B_OriginalImg/NI/Strong/P001/Anger/005.jpeg,005.jpeg,oulucasia-P001-Anger-005.jpg,P001,Anger,jpeg,cauca,6.0,-0.255,0.295,Anger,NaN,NaN,NaN,NaN,NaN,NaN,jpg
8199,oulucasia,nir,data/B_OriginalImg/NI/Strong/P001/Anger/001.jpeg,001.jpeg,oulucasia-P001-Anger-001.jpg,P001,Anger,jpeg,cauca,0.0,0.000,0.000,Neutral,NaN,NaN,NaN,NaN,NaN,NaN,jpg
8200,oulucasia,nir,data/B_OriginalImg/NI/Strong/P001/Anger/011.jpeg,011.jpeg,oulucasia-P001-Anger-011.jpg,P001,Anger,jpeg,cauca,6.0,-0.255,0.295,Anger,NaN,NaN,NaN,NaN,NaN,NaN,jpg
8201,oulucasia,nir,data/B_OriginalImg/NI/Strong/P001/Anger/008.jpeg,008.jpeg,oulucasia-P001-Anger-008.jpg,P001,Anger,jpeg,cauca,6.0,-0.255,0.295,Anger,NaN,NaN,NaN,NaN,NaN,NaN,jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10592,oulucasia,nir,data/B_OriginalImg/NI/Strong/P080/Surprise/012...,012.jpeg,oulucasia-P080-Surprise-012.jpg,P080,Surprise,jpeg,cauca,3.0,0.200,0.335,Surprise,NaN,NaN,NaN,NaN,NaN,NaN,jpg
10593,oulucasia,nir,data/B_OriginalImg/NI/Strong/P080/Surprise/018...,018.jpeg,oulucasia-P080-Surprise-018.jpg,P080,Surprise,jpeg,cauca,3.0,0.200,0.335,Surprise,NaN,NaN,NaN,NaN,NaN,NaN,jpg
10594,oulucasia,nir,data/B_OriginalImg/NI/Strong/P080/Surprise/010...,010.jpeg,oulucasia-P080-Surprise-010.jpg,P080,Surprise,jpeg,cauca,3.0,0.200,0.335,Surprise,NaN,NaN,NaN,NaN,NaN,NaN,jpg
10595,oulucasia,nir,data/B_OriginalImg/NI/Strong/P080/Surprise/023...,023.jpeg,oulucasia-P080-Surprise-023.jpg,P080,Surprise,jpeg,cauca,3.0,0.400,0.670,Surprise,NaN,NaN,NaN,NaN,NaN,NaN,jpg


,db,spectra,filepath,filename,adjusted_unique_filename,oulucasia-patient,oulucasia-emotion,filetype,race,expression,valence,arousal,expression_name,casia-session,casia-patient,casia-img_id,buaa-patient,buaa-image_type,customdb-patient,new_filetype
13732,casia,vis,data/CASIA_NISVIR/NIR-VIS/VIS/s1_VIS_00001_004...,s1_VIS_00001_004.jpg,casia-s1_VIS_00001_004.jpg,NaN,NaN,jpg,mongo,NaN,NaN,NaN,NaN,1.0,1.0,4.0,NaN,NaN,NaN,jpg
13733,casia,vis,data/CASIA_NISVIR/NIR-VIS/VIS/s1_VIS_00002_008...,s1_VIS_00002_008.jpg,casia-s1_VIS_00002_008.jpg,NaN,NaN,jpg,mongo,NaN,NaN,NaN,NaN,1.0,2.0,8.0,NaN,NaN,NaN,jpg
13734,casia,vis,data/CASIA_NISVIR/NIR-VIS/VIS/s1_VIS_00003_002...,s1_VIS_00003_002.jpg,casia-s1_VIS_00003_002.jpg,NaN,NaN,jpg,mongo,NaN,NaN,NaN,NaN,1.0,3.0,2.0,NaN,NaN,NaN,jpg
13735,casia,vis,data/CASIA_NISVIR/NIR-VIS/VIS/s1_VIS_00004_008...,s1_VIS_00004_008.jpg,casia-s1_VIS_00004_008.jpg,NaN,NaN,jpg,mongo,NaN,NaN,NaN,NaN,1.0,4.0,8.0,NaN,NaN,NaN,jpg
13736,casia,vis,data/CASIA_NISVIR/NIR-VIS/VIS/s1_VIS_00005_009...,s1_VIS_00005_009.jpg,casia-s1_VIS_00005_009.jpg,NaN,NaN,jpg,mongo,NaN,NaN,NaN,NaN,1.0,5.0,9.0,NaN,NaN,NaN,jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14461,casia,vis,data/CASIA_NISVIR/NIR-VIS/VIS/s4_VIS_30785_001...,s4_VIS_30785_001.jpg,casia-s4_VIS_30785_001.jpg,NaN,NaN,jpg,mongo,NaN,NaN,NaN,NaN,4.0,30785.0,1.0,NaN,NaN,NaN,jpg
14462,casia,vis,data/CASIA_NISVIR/NIR-VIS/VIS/s4_VIS_30786_002...,s4_VIS_30786_002.jpg,casia-s4_VIS_30786_002.jpg,NaN,NaN,jpg,mongo,NaN,NaN,NaN,NaN,4.0,30786.0,2.0,NaN,NaN,NaN,jpg
14463,casia,vis,data/CASIA_NISVIR/NIR-VIS/VIS/s4_VIS_30787_003...,s4_VIS_30787_003.jpg,casia-s4_VIS_30787_003.jpg,NaN,NaN,jpg,mongo,NaN,NaN,NaN,NaN,4.0,30787.0,3.0,NaN,NaN,NaN,jpg
14464,casia,vis,data/CASIA_NISVIR/NIR-VIS/VIS/s4_VIS_30836_003...,s4_VIS_30836_003.jpg,casia-s4_VIS_30836_003.jpg,NaN,NaN,jpg,mongo,NaN,NaN,NaN,NaN,4.0,30836.0,3.0,NaN,NaN,NaN,jpg


,db,spectra,filepath,filename,adjusted_unique_filename,oulucasia-patient,oulucasia-emotion,filetype,race,expression,valence,arousal,expression_name,casia-session,casia-patient,casia-img_id,buaa-patient,buaa-image_type,customdb-patient,new_filetype
12997,casia,nir,data/CASIA_NISVIR/NIR-VIS/NIR/s1_NIR_00001_004...,s1_NIR_00001_004.bmp,casia-s1_NIR_00001_004.jpg,NaN,NaN,bmp,mongo,NaN,NaN,NaN,NaN,1.0,1.0,4.0,NaN,NaN,NaN,jpg
12998,casia,nir,data/CASIA_NISVIR/NIR-VIS/NIR/s1_NIR_00002_001...,s1_NIR_00002_001.bmp,casia-s1_NIR_00002_001.jpg,NaN,NaN,bmp,mongo,NaN,NaN,NaN,NaN,1.0,2.0,1.0,NaN,NaN,NaN,jpg
12999,casia,nir,data/CASIA_NISVIR/NIR-VIS/NIR/s1_NIR_00003_010...,s1_NIR_00003_010.bmp,casia-s1_NIR_00003_010.jpg,NaN,NaN,bmp,mongo,NaN,NaN,NaN,NaN,1.0,3.0,10.0,NaN,NaN,NaN,jpg
13000,casia,nir,data/CASIA_NISVIR/NIR-VIS/NIR/s1_NIR_00004_007...,s1_NIR_00004_007.bmp,casia-s1_NIR_00004_007.jpg,NaN,NaN,bmp,mongo,NaN,NaN,NaN,NaN,1.0,4.0,7.0,NaN,NaN,NaN,jpg
13001,casia,nir,data/CASIA_NISVIR/NIR-VIS/NIR/s1_NIR_00005_008...,s1_NIR_00005_008.bmp,casia-s1_NIR_00005_008.jpg,NaN,NaN,bmp,mongo,NaN,NaN,NaN,NaN,1.0,5.0,8.0,NaN,NaN,NaN,jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13727,casia,nir,data/CASIA_NISVIR/NIR-VIS/NIR/s4_NIR_30785_008...,s4_NIR_30785_008.bmp,casia-s4_NIR_30785_008.jpg,NaN,NaN,bmp,mongo,NaN,NaN,NaN,NaN,4.0,30785.0,8.0,NaN,NaN,NaN,jpg
13728,casia,nir,data/CASIA_NISVIR/NIR-VIS/NIR/s4_NIR_30786_001...,s4_NIR_30786_001.bmp,casia-s4_NIR_30786_001.jpg,NaN,NaN,bmp,mongo,NaN,NaN,NaN,NaN,4.0,30786.0,1.0,NaN,NaN,NaN,jpg
13729,casia,nir,data/CASIA_NISVIR/NIR-VIS/NIR/s4_NIR_30787_016...,s4_NIR_30787_016.bmp,casia-s4_NIR_30787_016.jpg,NaN,NaN,bmp,mongo,NaN,NaN,NaN,NaN,4.0,30787.0,16.0,NaN,NaN,NaN,jpg
13730,casia,nir,data/CASIA_NISVIR/NIR-VIS/NIR/s4_NIR_30836_001...,s4_NIR_30836_001.bmp,casia-s4_NIR_30836_001.jpg,NaN,NaN,bmp,mongo,NaN,NaN,NaN,NaN,4.0,30836.0,1.0,NaN,NaN,NaN,jpg


,db,spectra,filepath,filename,adjusted_unique_filename,oulucasia-patient,oulucasia-emotion,filetype,race,expression,valence,arousal,expression_name,casia-session,casia-patient,casia-img_id,buaa-patient,buaa-image_type,customdb-patient,new_filetype


,db,spectra,filepath,filename,adjusted_unique_filename,oulucasia-patient,oulucasia-emotion,filetype,race,expression,valence,arousal,expression_name,casia-session,casia-patient,casia-img_id,buaa-patient,buaa-image_type,customdb-patient,new_filetype
1,buaa,nir,data/BUAA/BUAAVISNIR/19/2.bmp,2.bmp,buaa-19-2.jpg,NaN,NaN,bmp,mongo,0.0,0.00,0.00,Neutral,NaN,NaN,NaN,19,neutral,NaN,jpg
3,buaa,nir,data/BUAA/BUAAVISNIR/19/4.bmp,4.bmp,buaa-19-4.jpg,NaN,NaN,bmp,mongo,0.0,0.00,0.00,Neutral,NaN,NaN,NaN,19,tilt,NaN,jpg
5,buaa,nir,data/BUAA/BUAAVISNIR/19/6.bmp,6.bmp,buaa-19-6.jpg,NaN,NaN,bmp,mongo,0.0,0.00,0.00,Neutral,NaN,NaN,NaN,19,tilt,NaN,jpg
7,buaa,nir,data/BUAA/BUAAVISNIR/19/8.bmp,8.bmp,buaa-19-8.jpg,NaN,NaN,bmp,mongo,0.0,0.00,0.00,Neutral,NaN,NaN,NaN,19,tilt,NaN,jpg
9,buaa,nir,data/BUAA/BUAAVISNIR/19/10.bmp,10.bmp,buaa-19-10.jpg,NaN,NaN,bmp,mongo,0.0,0.00,0.00,Neutral,NaN,NaN,NaN,19,tilt,NaN,jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5929,buaa,nir,data/BUAA/BUAAVISNIR/42/18.bmp,18.bmp,buaa-42-18.jpg,NaN,NaN,bmp,mongo,0.0,0.00,0.00,Neutral,NaN,NaN,NaN,42,tilt,NaN,jpg
5931,buaa,nir,data/BUAA/BUAAVISNIR/42/22.bmp,22.bmp,buaa-42-22.jpg,NaN,NaN,bmp,mongo,1.0,0.93,0.12,Happy,NaN,NaN,NaN,42,affection,NaN,jpg
5933,buaa,nir,data/BUAA/BUAAVISNIR/42/24.bmp,24.bmp,buaa-42-24.jpg,NaN,NaN,bmp,mongo,6.0,-0.31,0.62,Anger,NaN,NaN,NaN,42,affection,NaN,jpg
5935,buaa,nir,data/BUAA/BUAAVISNIR/42/26.bmp,26.bmp,buaa-42-26.jpg,NaN,NaN,bmp,mongo,0.0,0.00,-0.91,Neutral,NaN,NaN,NaN,42,affection,NaN,jpg


,db,spectra,filepath,filename,adjusted_unique_filename,oulucasia-patient,oulucasia-emotion,filetype,race,expression,valence,arousal,expression_name,casia-session,casia-patient,casia-img_id,buaa-patient,buaa-image_type,customdb-patient,new_filetype
5952,customdb,nir,Custom_DB/custom_nir/2-3-surprise.jpg,2-3-surprise.jpg,customdb-2-3-surprise.jpg,NaN,NaN,jpg,cauca,4.0,-0.01,0.79,Fear,NaN,NaN,NaN,NaN,NaN,2.0,jpg
5953,customdb,nir,Custom_DB/custom_nir/13-5-anger.jpg,13-5-anger.jpg,customdb-13-5-anger.jpg,NaN,NaN,jpg,cauca,6.0,-0.42,0.76,Anger,NaN,NaN,NaN,NaN,NaN,13.0,jpg
5954,customdb,nir,Custom_DB/custom_nir/6-0-neutral.jpg,6-0-neutral.jpg,customdb-6-0-neutral.jpg,NaN,NaN,jpg,cauca,0.0,0.01,0.02,Neutral,NaN,NaN,NaN,NaN,NaN,6.0,jpg
5955,customdb,nir,Custom_DB/custom_nir/7-7-contempt_.jpg,7-7-contempt_.jpg,customdb-7-7-contempt_.jpg,NaN,NaN,jpg,cauca,7.0,-0.58,0.63,Contempt,NaN,NaN,NaN,NaN,NaN,7.0,jpg
5956,customdb,nir,Custom_DB/custom_nir/1-10-calm.jpg,1-10-calm.jpg,customdb-1-10-calm.jpg,NaN,NaN,jpg,cauca,0.0,0.22,-0.29,Neutral,NaN,NaN,NaN,NaN,NaN,1.0,jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6106,customdb,nir,Custom_DB/custom_nir/10-0-neutral.jpg,10-0-neutral.jpg,customdb-10-0-neutral.jpg,NaN,NaN,jpg,cauca,0.0,-0.02,-0.03,Neutral,NaN,NaN,NaN,NaN,NaN,10.0,jpg
6107,customdb,nir,Custom_DB/custom_nir/13-6-contempt.jpg,13-6-contempt.jpg,customdb-13-6-contempt.jpg,NaN,NaN,jpg,cauca,7.0,-0.55,0.64,Contempt,NaN,NaN,NaN,NaN,NaN,13.0,jpg
6108,customdb,nir,Custom_DB/custom_nir/8-7-contempt.jpg,8-7-contempt.jpg,customdb-8-7-contempt.jpg,NaN,NaN,jpg,cauca,7.0,0.00,0.63,Contempt,NaN,NaN,NaN,NaN,NaN,8.0,jpg
6109,customdb,nir,Custom_DB/custom_nir/1-9-disgust.jpg,1-9-disgust.jpg,customdb-1-9-disgust.jpg,NaN,NaN,jpg,cauca,5.0,-0.67,0.49,Disgust,NaN,NaN,NaN,NaN,NaN,1.0,jpg


## Create split for CycleGAN - latest version

The whole merged dataset will comprise of:
<!-- | Database            | #NIR   | #VIS   | #NIR mongo | #NIR cauca | #VIS mongo | #VIS cauca | #affected             |
|---------------------|--------|--------|------------|------------|------------|------------|-----------------------|
| Oulu-Casia          | 10379 | 10426 | 4170(40%)  | 6255(60%)  | 4170(40%)  | 6255(60%)  | yes                   |
| CASIA               | 12487  | 5093  | 12487      | 0          | 5093      | 0          | no                    |
| BUAA                | 1950   | 0      | 1950       | 0          | 0          | 0          | 1/3                  |
| AffectNet| 0      | 9503   |  0        | 0          | 1930(~10%)     | 7723(80%)  | yes                   |
| Custom DB           | 159    | 0      | 0          | 159        | 0          | 0          | yes                   |
| **Total**           | **24975**  | **24975**   | **18756** (75%)| **6405** (25%)| **11193** (45%)| **13979** (55%)| **-**                 |

 -->

In [62]:
# There will be only train and test split. 
# Test split will be containing patients, that were not in the train split.
SEED = 0
fraction = 1/10

df_aff = df_cyclegan_all[df_cyclegan_all["db"] == "affectnet"]
random.seed(SEED)
df_aff_test = df_aff.sample(int(fraction * df_aff.shape[0]))

df_buaa = df_cyclegan_all[(df_cyclegan_all["db"] == "buaa")]
random.seed(SEED)
df_buaa_test_patients = random.sample(list(df_buaa["buaa-patient"].unique()), int(fraction * df_buaa["buaa-patient"].nunique()))
df_buaa_test = df_buaa[df_buaa["buaa-patient"].isin(df_buaa_test_patients)]

df_casia_vis = df_cyclegan_all[(df_cyclegan_all["db"] == "casia") & (df_cyclegan_all["spectra"]=="vis")]
random.seed(SEED)
df_casia_vis_test = df_casia_vis.sample(int(fraction * df_casia_vis.shape[0]))

df_casia_nir = df_cyclegan_all[(df_cyclegan_all["db"] == "casia") & (df_cyclegan_all["spectra"]=="nir")]
random.seed(SEED)
df_casia_nir_test = df_casia_nir.sample(int(fraction * df_casia_nir.shape[0]))

df_customdb = df_cyclegan_all[df_cyclegan_all["db"] == "customdb"]
random.seed(SEED)
df_customdb_test_patients = random.sample(list(df_customdb["customdb-patient"].unique()), int(fraction * df_customdb["customdb-patient"].nunique()))
df_customdb_test = df_customdb[df_customdb["customdb-patient"].isin(df_customdb_test_patients)]

df_oulucasia = df_cyclegan_all[df_cyclegan_all["db"] == "oulucasia"]
random.seed(SEED)
df_oulucasia_test_patients = random.sample(list(df_oulucasia["oulucasia-patient"].unique()), int(fraction * df_oulucasia["oulucasia-patient"].nunique()))
df_oulucasia_test = df_oulucasia[df_oulucasia["oulucasia-patient"].isin(df_oulucasia_test_patients)]

df_cyclegan_test = pd.concat([df_aff_test, df_buaa_test, df_casia_vis_test, df_casia_nir_test, df_customdb_test, df_oulucasia_test])

nir_count = len(df_cyclegan_test[df_cyclegan_test['spectra'] == 'nir'])
vis_rows = df_cyclegan_test[df_cyclegan_test['spectra'] == 'vis']
drop_indices = np.random.choice(vis_rows.index, size=len(vis_rows)-nir_count, replace=False)
df_cyclegan_test = df_cyclegan_test.drop(drop_indices)

df_cyclegan_train = df_cyclegan_all.drop(df_cyclegan_test.index)

nir_count = len(df_cyclegan_train[df_cyclegan_train['spectra'] == 'nir'])
vis_rows = df_cyclegan_train[df_cyclegan_train['spectra'] == 'vis']
drop_indices = np.random.choice(vis_rows.index, size=len(vis_rows)-nir_count, replace=False)
df_cyclegan_train = df_cyclegan_train.drop(drop_indices)

# write the affiliation of image to split into the dataframe with all annotations
df_cyclegan_all['split'] = None
df_cyclegan_all.loc[df_cyclegan_all['adjusted_unique_filename'].isin(df_cyclegan_test['adjusted_unique_filename']), 'split'] = 'test'
df_cyclegan_all.loc[df_cyclegan_all['adjusted_unique_filename'].isin(df_cyclegan_train['adjusted_unique_filename']), 'split'] = 'train'

Display overview of the splits

In [63]:
display(get_overview(df_cyclegan_train))
display(get_overview(df_cyclegan_test))

,#NIR,#VIS,#NIR mongo,#NIR cauca,#VIS mongo,#VIS cauca
db,,,,,,
affectnet,0,1884,0,0,0,1884
buaa,1739,0,1739,0,0,0
casia,662,664,662,0,664,0
customdb,151,0,0,151,0,0
oulucasia,2160,2164,840,1320,841,1323
Total,4712,4712,3241,1471,1505,3207


,#NIR,#VIS,#NIR mongo,#NIR cauca,#VIS mongo,#VIS cauca
db,,,,,,
affectnet,0,202,0,0,0,202
buaa,187,0,187,0,0,0
casia,73,70,73,0,70,0
customdb,8,0,0,8,0,0
oulucasia,240,236,120,120,119,117
Total,508,508,380,128,189,319


Save the splits

In [64]:
df_cyclegan_all.to_csv('df_cyclegan_all_2.csv')
df_cyclegan_train.to_csv('df_cyclegan_train_2.csv')
df_cyclegan_test.to_csv('df_cyclegan_test_2.csv')

In [65]:
# a = df_cyclegan_all[df_cyclegan_all["split"]=="train"]["adjusted_unique_filename"].to_list()
# b = df_cyclegan_train[df_cyclegan_train["spectra"]=="vis"]["adjusted_unique_filename"].to_list()
# print(len(a))
# print(len(set(a) - set(b)))

In [66]:
# print(len(df_aff_test.index))
# print(len(df_buaa_test.index))
# print(len(df_casia_vis_test.index))
# print(len(df_casia_nir_test.index))
# print(len(df_customdb_test.index))
# print(len(df_oulucasia_test.index))
# print("-"*20)
# print(len(df_cyclegan_test.index))
# print(len(df_cyclegan_test[df_cyclegan_test["spectra"]=="nir"].index))
# print(len(df_cyclegan_test[df_cyclegan_test["spectra"]=="vis"].index))
# print("-"*20)
# print(len(df_cyclegan_train.index))
# print(len(df_cyclegan_train[df_cyclegan_train["spectra"]=="nir"]))
# print(len(df_cyclegan_train[df_cyclegan_train["spectra"]=="vis"]))

Process the images - 

In [16]:
from skeleton.inference import Inference, CenterFace
import cv2

models = {
    "face_detector": {
        "net_type": Inference.net_type.FACE_DETECTOR_RETINAFACE,
        "remove_black_stripes": False,
        "display_images": True,
        "save_image_to_folder": None,
    },
    "spectrum_translator": {
        "net_type": None, 
        "pth_to_onnx": 'models/cyclegan-snellius-casia-oulucasia-double_gen_op/NIR2VIS_cyclegan_snellius_casia_oulucasia_double_gen_opt-GA-20epoch.onnx',
        "input_as_avg_grayscale": False,
        "output_as_avg_grayscale": True,
    },
    "fer": {
        "net_type": None,
        "pth_to_onnx": "models/mobilenet_NIR/mobilenet_on_AffectNet-NIR/mobilenet_aff_nir-aff_continue.onnx",
        "va_to_circumplex_model": False,
    }
}
inf = Inference(models, None, verbose=True, debug=True)

Using '{'net_type': <net_type.FACE_DETECTOR_RETINAFACE: 'G'>, 'remove_black_stripes': False, 'display_images': True}' as face detector model


Preprocess all data for 

In [17]:
import pandas as pd
import os
import cv2
df_cyclegan_test = pd.read_csv('df_cyclegan_test.csv')
df_cyclegan_train = pd.read_csv('df_cyclegan_train.csv')

In [ ]:
nir_fps = df_cyclegan_test[(df_cyclegan_test["spectra"]=="nir")].filepath.to_list()
nir_target_fps = [str(os.path.join("nir_test", fp)) for fp in df_cyclegan_test[df_cyclegan_test["spectra"]=="nir"].adjusted_unique_filename.to_list()]

nir_out = inf.infer_instant_from_filenames(nir_fps)
for v, fp in zip(nir_out, nir_target_fps):
    if fp == 'data/BUAA/BUAAVISNIR/72/2.bmp':
        print('s')
        continue
    cv2.imwrite(fp.split('.')[0]+".jpg", v[0])

In [ ]:
nir_fps = df_cyclegan_train[(df_cyclegan_train["spectra"]=="nir")].filepath.to_list()
nir_target_fps = [str(os.path.join("nir_train", fp)) for fp in df_cyclegan_train[df_cyclegan_train["spectra"]=="nir"].adjusted_unique_filename.to_list()]

nir_out = inf.infer_instant_from_filenames(nir_fps)
for v, fp in zip(nir_out, nir_target_fps):
    cv2.imwrite(fp, v[0])

In [ ]:
# vis_df = df_cyclegan_test[((df_cyclegan_test["spectra"]=="vis")&(df_cyclegan_test["db"]=="casia"))]
# vis_fps = vis_df.filepath.to_list()
# vis_target_fps = [str(os.path.join("vis_test", fp)) for fp in vis_df.adjusted_unique_filename.to_list()]

# vis_out = inf.infer_instant_from_filenames(vis_fps)
# for v, fp_tgt in zip(vis_out, vis_target_fps):
#     v[0] = cv2.cvtColor(v[0], cv2.COLOR_RGB2BGR)
#     cv2.imwrite(fp_tgt, v[0])
    
df_affectnet_vl = get_files_affectnet('data/AffectNet-8Labels/train_set/images/', 439, labels_fp="data/AffectNet-8Labels/train_custom.csv")
vis_df = df_affectnet_vl[((df_affectnet_vl["spectra"]=="vis"))]
vis_fps = vis_df.filepath.to_list()
vis_target_fps = [str(os.path.join("vis_test", fp)) for fp in vis_df.adjusted_unique_filename.to_list()]

vis_out = inf.infer_instant_from_filenames(vis_fps)
for v, fp_tgt in zip(vis_out, vis_target_fps):
    v[0] = cv2.cvtColor(v[0], cv2.COLOR_RGB2BGR)
    cv2.imwrite(fp_tgt, v[0])

# vis_df = df_cyclegan_test[((df_cyclegan_test["spectra"]=="vis")&(df_cyclegan_test["db"]!="affectnet"))]
# vis_fps = vis_df.filepath.to_list()
# vis_target_fps = [str(os.path.join("vis_test", fp)) for fp in vis_df.adjusted_unique_filename.to_list()]

# vis_out = inf.infer_instant_from_filenames(vis_fps)
# for v, fp_tgt in zip(vis_out, vis_target_fps):
#     v[0] = cv2.cvtColor(v[0], cv2.COLOR_RGB2BGR)
#     cv2.imwrite(fp_tgt, v[0])
    
# import shutil

# fps_aff = df_cyclegan_test[df_cyclegan_test["db"]=='affectnet'].filepath.to_list()
# tgt_fps_aff = [str(os.path.join("vis_test", fn)) for fn in df_cyclegan_test[df_cyclegan_test["db"]=='affectnet'].adjusted_unique_filename.to_list()]

# for src, dst in zip(fps_aff, tgt_fps_aff):
#     shutil.copy(src, dst)

Add stripes to AffectNet

In [2]:
import numpy as np
from PIL import Image, ImageDraw
import random
import threading
import os

def add_black_stripes(image_path, stripe_width=40):
    try:
        # Open the image
        original_image = Image.open(image_path)

        # Get image dimensions
        width, height = original_image.size

        # Create a new image with the same size
        new_image = Image.new("RGB", (width, height), color="black")

        # Paste the original image in the center
        new_image.paste(original_image, (0, 0))

        # Draw black stripes on the left and right borders
        draw = ImageDraw.Draw(new_image)
        draw.rectangle([0, 0, stripe_width, height], fill="black")  # Left stripe
        draw.rectangle([width - stripe_width, 0, width, height], fill="black")  # Right stripe

        # Convert the image to a NumPy array
        image_array = np.array(new_image)

        return image_array

    except Exception as e:
        print(f"Error: {e}")
        return None
    
def process_one_thread(thread_id, paths):
    for i, path in enumerate(paths):
        print(f"T{thread_id}:: #{i}")
        try:
            fp = os.path.join("data/AffectNet-8Labels/val_set/images/", path)
            random_number = random.randint(25, 45)
            striped_image_array = add_black_stripes(fp, stripe_width=random_number)
            
            # Save the modified image
            Image.fromarray(striped_image_array).save(os.path.join("data/AffectNet-8Labels/val_set_stripped/", path))
        except:
            print("Error at", path)

    
train_images = os.listdir('data/AffectNet-8Labels/val_set/images/')
len_train = len(train_images)
print("loaded", len_train)
len_qt = len_train // 4
train_pt1 = train_images[:len_qt]
train_pt2 = train_images[len_qt:2*len_qt]
train_pt3 = train_images[2*len_qt:3*len_qt]
train_pt4 = train_images[3*len_qt:]
print("part lengths:", len(train_pt1),len(train_pt2),len(train_pt3),len(train_pt4))

# create threads
t1 = threading.Thread(target=process_one_thread, args=(0, train_pt1))
t2 = threading.Thread(target=process_one_thread, args=(1, train_pt2))
t3 = threading.Thread(target=process_one_thread, args=(2, train_pt3))
t4 = threading.Thread(target=process_one_thread, args=(3, train_pt4))

# start threads
t1.start()
t2.start()
t3.start()
t4.start()

# wait for all threads to finish
t1.join()
t2.join()
t3.join()
t4.join()
    
# # Example usage
# image_path = "data/AffectNet-8Labels/train_set/images/26661.jpg"
# striped_image_array = add_black_stripes(image_path)
# display(Image.fromarray(striped_image_array))

loaded 3999
part lengths: 999 999 999 1002
T0:: #0
T1:: #0
T2:: #0
T3:: #0
T0:: #1
T2:: #1
T1:: #1
T3:: #1
T0:: #2
T2:: #2
T3:: #2
T2:: #3
T3:: #3
T2:: #4
T3:: #4
T2:: #5
T1:: #2
T0:: #3
T2:: #6
T0:: #4
T2:: #7
T3:: #5
T0:: #5
T2:: #8
T0:: #6
T1:: #3
T0:: #7
T3:: #6
T2:: #9
T3:: #7
T2:: #10
T1:: #4
T0:: #8
T1:: #5
T2:: #11
T3:: #8
T1:: #6
T2:: #12
T1:: #7
T2:: #13
T2:: #14
T1:: #8
T0:: #9
T3:: #9
T3:: #10
T1:: #9
T2:: #15
T3:: #11
T1:: #10
T0:: #10
T3:: #12
T0:: #11
T3:: #13
T3:: #14
T1:: #11
T0:: #12
T3:: #15
T0:: #13
T2:: #16
T3:: #16
T2:: #17
T1:: #12
T1:: #13
T2:: #18
T0:: #14
T1:: #14
T1:: #15
T2:: #19
T0:: #15
T1:: #16
T3:: #17
T1:: #17
T3:: #18
T0:: #16
T1:: #18
T0:: #17
T3:: #19
T2:: #20
T3:: #20
T3:: #21
T0:: #18
T1:: #19
T2:: #21
T3:: #22
T1:: #20
T3:: #23
T0:: #19
T2:: #22
T3:: #24
T0:: #20
T1:: #21
T3:: #25
T0:: #21
T2:: #23
T2:: #24
T0:: #22
T1:: #22
T0:: #23
T1:: #23
T2:: #25
T3:: #26
T2:: #26
T0:: #24
T1:: #24
T2:: #27
T3:: #27
T0:: #25
T3:: #28
T0:: #26
T1:: #25
T2:: #2

T3:: #219
T1:: #215
T3:: #220
T3:: #221
T2:: #226
T3:: #222
T3:: #223
T1:: #216
T2:: #227
T2:: #228
T0:: #202
T2:: #229
T1:: #217
T2:: #230
T3:: #224
T2:: #231
T2:: #232
T1:: #218
T1:: #219
T3:: #225
T0:: #203
T2:: #233
T1:: #220
T0:: #204
T0:: #205
T1:: #221
T2:: #234
T1:: #222
T3:: #226
T1:: #223
T1:: #224
T0:: #206
T2:: #235
T2:: #236
T1:: #225
T2:: #237
T0:: #207
T3:: #227
T1:: #226
T0:: #208
T1:: #227
T1:: #228
T3:: #228
T2:: #238
T1:: #229
T0:: #209
T2:: #239
T2:: #240
T3:: #229
T1:: #230
T3:: #230
T0:: #210
T3:: #231
T1:: #231
T3:: #232
T0:: #211
T2:: #241
T2:: #242
T2:: #243
T1:: #232
T3:: #233
T1:: #233
T2:: #244
T0:: #212
T0:: #213
T3:: #234
T0:: #214
T1:: #234
T1:: #235
T0:: #215
T1:: #236
T0:: #216
T3:: #235
T3:: #236
T2:: #245
T1:: #237
T3:: #237
T0:: #217
T2:: #246
T2:: #247
T0:: #218
T3:: #238
T1:: #238
T1:: #239
T0:: #219
T3:: #239
T2:: #248
T0:: #220
T2:: #249
T1:: #240
T3:: #240
T1:: #241
T3:: #241
T2:: #250
T0:: #221
T2:: #251
T0:: #222
T3:: #242
T1:: #242
T0:: #223


T0:: #422
T1:: #443
T2:: #403
T3:: #440
T1:: #444
T0:: #423
T2:: #404
T3:: #441
T0:: #424
T2:: #405
T1:: #445
T0:: #425
T2:: #406
T1:: #446
T3:: #442
T2:: #407
T1:: #447
T0:: #426
T3:: #443
T2:: #408
T2:: #409
T1:: #448
T3:: #444
T0:: #427
T0:: #428
T3:: #445
T1:: #449
T2:: #410
T0:: #429
T1:: #450
T2:: #411
T0:: #430
T3:: #446
T3:: #447
T1:: #451
T2:: #412
T0:: #431
T1:: #452
T2:: #413
T1:: #453
T3:: #448
T2:: #414
T0:: #432
T1:: #454
T3:: #449
T1:: #455
T0:: #433
T2:: #415
T1:: #456
T3:: #450
T2:: #416
T0:: #434
T3:: #451
T1:: #457
T2:: #417
T3:: #452
T1:: #458
T0:: #435
T3:: #453
T1:: #459
T2:: #418
T2:: #419
T1:: #460
T0:: #436
T3:: #454
T2:: #420
T1:: #461
T3:: #455
T0:: #437
T0:: #438
T2:: #421
T2:: #422
T1:: #462
T0:: #439
T0:: #440
T1:: #463
T0:: #441
T3:: #456
T2:: #423
T2:: #424
T3:: #457
T0:: #442
T1:: #464
T2:: #425
T1:: #465
T2:: #426
T0:: #443
T3:: #458
T2:: #427
T0:: #444
T2:: #428
T2:: #429
T3:: #459
T0:: #445
T2:: #430
T1:: #466
T3:: #460
T2:: #431
T1:: #467
T0:: #446


T0:: #634
T1:: #646
T0:: #635
T2:: #622
T2:: #623
T1:: #647
T0:: #636
T2:: #624
T1:: #648
T1:: #649
T0:: #637
T2:: #625
T3:: #632
T2:: #626
T3:: #633
T2:: #627
T1:: #650
T3:: #634
T2:: #628
T0:: #638
T2:: #629
T0:: #639
T2:: #630
T2:: #631
T1:: #651
T0:: #640
T2:: #632
T1:: #652
T2:: #633
T0:: #641
T3:: #635
T2:: #634
T1:: #653
T3:: #636
T1:: #654
T2:: #635
T2:: #636
T0:: #642
T2:: #637
T3:: #637
T1:: #655
T1:: #656
T0:: #643
T2:: #638
T3:: #638
T0:: #644
T2:: #639
T3:: #639
T1:: #657
T0:: #645
T1:: #658
T2:: #640
T3:: #640
T1:: #659
T0:: #646
T0:: #647
T0:: #648
T2:: #641
T1:: #660
T0:: #649
T2:: #642
T0:: #650
T1:: #661
T3:: #641
T2:: #643
T0:: #651
T2:: #644
T3:: #642
T3:: #643
T1:: #662
T2:: #645
T1:: #663
T3:: #644
T2:: #646
T3:: #645
T1:: #664
T3:: #646
T1:: #665
T2:: #647T1:: #666

T0:: #652
T2:: #648
T0:: #653
T1:: #667
T0:: #654
T1:: #668
T0:: #655
T3:: #647
T0:: #656
T1:: #669
T3:: #648
T0:: #657
T2:: #649
T0:: #658
T3:: #649
T2:: #650
T1:: #670
T3:: #650
T3:: #651
T2:: #651


T3:: #823
T1:: #858
T2:: #842
T1:: #859
T0:: #833
T3:: #824
T0:: #834
T0:: #835
T1:: #860
T2:: #843
T1:: #861
T3:: #825
T1:: #862
T2:: #844
T0:: #836
T1:: #863
T2:: #845
T3:: #826
T2:: #846
T0:: #837
T1:: #864
T2:: #847
T0:: #838
T1:: #865
T3:: #827
T3:: #828
T2:: #848
T0:: #839
T1:: #866
T3:: #829
T1:: #867
T0:: #840
T2:: #849
T1:: #868
T0:: #841
T1:: #869
T2:: #850
T1:: #870
T3:: #830
T0:: #842
T3:: #831
T2:: #851
T0:: #843
T2:: #852
T0:: #844
T1:: #871
T3:: #832
T2:: #853
T3:: #833
T3:: #834
T1:: #872
T0:: #845
T3:: #835
T1:: #873
T2:: #854
T3:: #836
T2:: #855
T3:: #837
T2:: #856
T1:: #874
T0:: #846
T1:: #875
T0:: #847
T3:: #838
T1:: #876
T0:: #848
T2:: #857
T0:: #849
T0:: #850
T3:: #839
T1:: #877
T2:: #858
T1:: #878
T2:: #859
T0:: #851
T1:: #879
T2:: #860
T1:: #880
T2:: #861
T3:: #840
T1:: #881
T0:: #852
T1:: #882
T3:: #841
T0:: #853
T3:: #842
T1:: #883
T1:: #884
T0:: #854
T2:: #862
T0:: #855
T3:: #843
T0:: #856
T2:: #863
T1:: #885
T0:: #857
T0:: #858
T2:: #864
T1:: #886
T0:: #859


In [40]:
df_affectnet_vl

,expression,valence,arousal,filetype,filename,expression_id,filepath,adjusted_unique_filename,db,spectra,race
111072,Happy,0.677538,0.042107,jpg,198560.jpg,1,data/AffectNet-8Labels/train_set/images/198560...,affectnet-198560.jpg,affectnet,vis,cauca
237882,Happy,0.857143,-0.095238,jpg,64990.jpg,1,data/AffectNet-8Labels/train_set/images/64990.jpg,affectnet-64990.jpg,affectnet,vis,cauca
57482,Neutral,0.004313,-0.043133,jpg,5535.jpg,0,data/AffectNet-8Labels/train_set/images/5535.jpg,affectnet-5535.jpg,affectnet,vis,cauca
246612,Happy,0.716221,-0.179055,jpg,197832.jpg,1,data/AffectNet-8Labels/train_set/images/197832...,affectnet-197832.jpg,affectnet,vis,cauca
182199,Neutral,-0.111259,0.100134,jpg,129158.jpg,0,data/AffectNet-8Labels/train_set/images/129158...,affectnet-129158.jpg,affectnet,vis,cauca
...,...,...,...,...,...,...,...,...,...,...,...
91852,Happy,0.534852,-0.081953,jpg,395172.jpg,1,data/AffectNet-8Labels/train_set/images/395172...,affectnet-395172.jpg,affectnet,vis,cauca
152596,Happy,0.428571,-0.071429,jpeg,196315.jpg,1,data/AffectNet-8Labels/train_set/images/196315...,affectnet-196315.jpg,affectnet,vis,cauca
137006,Neutral,0.064700,0.000000,jpg,229734.jpg,0,data/AffectNet-8Labels/train_set/images/229734...,affectnet-229734.jpg,affectnet,vis,cauca
192533,Anger,-0.488290,0.622245,jpg,391785.jpg,6,data/AffectNet-8Labels/train_set/images/391785...,affectnet-391785.jpg,affectnet,vis,cauca


Something idk what

In [ ]:
vis_df = df_cyclegan_train[((df_cyclegan_train["spectra"]=="vis")&(df_cyclegan_train["db"]=="casia"))]
vis_fps = vis_df.filepath.to_list()
vis_target_fps = [str(os.path.join("vis_train", fp)) for fp in vis_df.adjusted_unique_filename.to_list()]

vis_out = inf.infer_instant_from_filenames(vis_fps)
for v, fp_tgt in zip(vis_out, vis_target_fps):
    v[0] = cv2.cvtColor(v[0], cv2.COLOR_RGB2BGR)
    cv2.imwrite(fp_tgt, v[0])
    
# import shutil

# fps_aff = df_cyclegan_train[df_cyclegan_train["db"]=='affectnet'].filepath.to_list()
# tgt_fps_aff = [str(os.path.join("vis_train", fn)) for fn in df_cyclegan_train[df_cyclegan_train["db"]=='affectnet'].adjusted_unique_filename.to_list()]

# for src, dst in zip(fps_aff, tgt_fps_aff):
#     shutil.copy(src, dst)

# vis_df = df_cyclegan_train[((df_cyclegan_train["spectra"]=="vis")&(df_cyclegan_train["db"]!="affectnet"))]
# vis_fps = vis_df.filepath.to_list()
# vis_target_fps = [str(os.path.join("vis_train", fp)) for fp in vis_df.adjusted_unique_filename.to_list()]

# vis_out = inf.infer_instant_from_filenames(vis_fps)
# for v, fp_tgt in zip(vis_out, vis_target_fps):
#     v[0] = cv2.cvtColor(v[0], cv2.COLOR_RGB2BGR)
#     cv2.imwrite(fp_tgt, v[0])
    
# import shutil

# fps_aff = df_cyclegan_train[df_cyclegan_train["db"]=='affectnet'].filepath.to_list()
# tgt_fps_aff = [str(os.path.join("vis_train", fn)) for fn in df_cyclegan_train[df_cyclegan_train["db"]=='affectnet'].adjusted_unique_filename.to_list()]

# for src, dst in zip(fps_aff, tgt_fps_aff):
#     shutil.copy(src, dst)

## <span style="color:red;">TODO</span>: Create split for FER

The whole merged dataset comprise of:

...

In [5]:
import pandas

df_all = pd.read_csv('df_cyclegan_train.csv')

In [6]:
df_fer_all = df_all[(df_all["spectra"]=="nir") & (df_all["expression"].notna())]
df_fer_all.groupby(by=df_fer_all["db"]).count()

,Unnamed: 0,spectra,filepath,filename,adjusted_unique_filename,oulucasia-patient,oulucasia-emotion,filetype,race,expression,valence,arousal,casia-session,casia-patient,casia-img_id,buaa-patient,buaa-image_type,customdb-patient
db,,,,,,,,,,,,,,,,,,
buaa,1739,1739,1739,1739,1739,0,0,1739,1739,1739,1739,1739,0,0,0,1739,1739,0
customdb,151,151,151,151,151,0,0,151,151,151,151,151,0,0,0,0,0,151
oulucasia,2160,2160,2160,2160,2160,2160,2160,2160,2160,2160,2160,2160,0,0,0,0,0,0


In [10]:
d = df_fer_all[df_fer_all['db']=='customdb']
d.groupby(by="expression").count()

,Unnamed: 0,db,spectra,filepath,filename,adjusted_unique_filename,oulucasia-patient,oulucasia-emotion,filetype,race,valence,arousal,casia-session,casia-patient,casia-img_id,buaa-patient,buaa-image_type,customdb-patient
expression,,,,,,,,,,,,,,,,,,
Anger,16,16,16,16,16,16,0,0,16,16,16,16,0,0,0,0,0,16
Contempt,32,32,32,32,32,32,0,0,32,32,32,32,0,0,0,0,0,32
Disgust,13,13,13,13,13,13,0,0,13,13,13,13,0,0,0,0,0,13
Fear,15,15,15,15,15,15,0,0,15,15,15,15,0,0,0,0,0,15
Happy,18,18,18,18,18,18,0,0,18,18,18,18,0,0,0,0,0,18
Neutral,25,25,25,25,25,25,0,0,25,25,25,25,0,0,0,0,0,25
Sad,20,20,20,20,20,20,0,0,20,20,20,20,0,0,0,0,0,20
Surprise,12,12,12,12,12,12,0,0,12,12,12,12,0,0,0,0,0,12


## create-unpaired_casia_oulu-casia_data
### Create unpaired split
From Oulu-Casia and Casia2

In [ ]:
import json
from sklearn.model_selection import train_test_split
from pathlib import Path
import random

# Specify the directories
nir_path = Path('data/CASIA_NISVIR/NIR-VIS/NIR/')
vis_path = Path('data/CASIA_NISVIR/NIR-VIS/VIS/')

# Get the contents of the directories
nir_images = list(nir_path.glob('*.bmp'))  # replace with your image extension
vis_images = list(vis_path.glob('*.jpg'))  # replace with your image extension

# Determine the size of the smaller dataset
min_size = min(len(nir_images), len(vis_images))

# Randomly select 'min_size' images from both NIR and VIS datasets
nir_images_selected = random.sample(nir_images, min_size)
vis_images_selected = random.sample(vis_images, min_size)

# Get the images that are not selected
nir_images_unselected = list(set(nir_images) - set(nir_images_selected))
vis_images_unselected = list(set(vis_images) - set(vis_images_selected))

# Split the NIR images into train, val, and test sets
nir_train_val, nir_test = train_test_split(nir_images_selected, test_size=0.2, random_state=42)
nir_train, nir_val = train_test_split(nir_train_val, test_size=0.25, random_state=42)

# Split the VIS images into train, val, and test sets
vis_train_val, vis_test = train_test_split(vis_images_selected, test_size=0.2, random_state=42)
vis_train, vis_val = train_test_split(vis_train_val, test_size=0.25, random_state=42)

# Convert the Path objects to strings
nir_train = [str(path) for path in nir_train]
nir_val = [str(path) for path in nir_val]
nir_test = [str(path) for path in nir_test]
vis_train = [str(path) for path in vis_train]
vis_val = [str(path) for path in vis_val]
vis_test = [str(path) for path in vis_test]
nir_images_unselected = [str(path) for path in nir_images_unselected]
vis_images_unselected = [str(path) for path in vis_images_unselected]

# Prepare the metadata
metadata = {
    'nir_images_selected': len(nir_images_selected),
    'vis_images_selected': len(vis_images_selected),
    'nir_images_unselected': len(nir_images_unselected),
    'vis_images_unselected': len(vis_images_unselected),
    'nir_train': len(nir_train),
    'nir_val': len(nir_val),
    'nir_test': len(nir_test),
    'vis_train': len(vis_train),
    'vis_val': len(vis_val),
    'vis_test': len(vis_test),
}

# Prepare the data to be stored in the JSON file
data = {
    'nir_train': nir_train,
    'nir_val': nir_val,
    'nir_test': nir_test,
    'nir_rest': nir_images_unselected,
    'vis_train': vis_train,
    'vis_val': vis_val,
    'vis_test': vis_test,
    'vis_rest': vis_images_unselected,
    'metadata': metadata,
}

# Write the data to the JSON file
with open('splits/new/casia2_data_splits.json', 'w') as f:
    json.dump(data, f)


In [ ]:
from skeleton.data.splitter import DatasetSplitter
import json
import os

# define splitter
splitter = DatasetSplitter(
    vl_data_path=Path('data/B_OriginalImg/VL/Strong/'),
    ni_data_path=Path('data/B_OriginalImg/NI/Strong/'),
    train_n_img_picked=5,
    test_n_img_picked=2,
    val_n_img_picked=2,
    json_train_split_pth=Path('_train_tmp.json'),
    json_test_split_pth=Path('_test_tmp.json'),
    json_val_split_pth=Path('_val_tmp.json'),
)

# split the files
splitter()


# Load the temporary JSON files
with open('_train_tmp.json', 'r') as f:
    train_data = json.load(f)
with open('_test_tmp.json', 'r') as f:
    test_data = json.load(f)
with open('_val_tmp.json', 'r') as f:
    val_data = json.load(f)

# Prepare the metadata
metadata = {
    'nir_images': len(train_data['ni']) + len(test_data['ni']) + len(val_data['ni']),
    'vis_images': len(train_data['vl']) + len(test_data['vl']) + len(val_data['vl']),
    'nir_train': len(train_data['ni']),
    'nir_val': len(val_data['ni']),
    'nir_test': len(test_data['ni']),
    'vis_train': len(train_data['vl']),
    'vis_val': len(val_data['vl']),
    'vis_test': len(test_data['vl']),
}

# Prepare the data to be stored in the JSON file
data = {
    'nir_train': train_data['ni'],
    'nir_val': val_data['ni'],
    'nir_test': test_data['ni'],
    'vis_train': train_data['vl'],
    'vis_val': val_data['vl'],
    'vis_test': test_data['vl'],
    'metadata': metadata,
}

# Write the data to the JSON file
with open('splits/new/oulucasia_data_splits.json', 'w') as f:
    json.dump(data, f)

# Remove the temporary files
os.remove('_train_tmp.json')
os.remove('_test_tmp.json')
os.remove('_val_tmp.json')


In [ ]:
# Load the existing data from the two files
with open('splits/new/casia2_data_splits.json', 'r') as f:
    casia2_data = json.load(f)
with open('splits/new/oulucasia_data_splits.json', 'r') as f:
    oulucasia_data = json.load(f)

# Merge the data
merged_data = {}
for key in set(casia2_data.keys()).union(oulucasia_data.keys()):
    if key != 'metadata':
        merged_data[key] = casia2_data.get(key, []) + oulucasia_data.get(key, [])

# Merge the metadata
merged_metadata = {}
for key in set(casia2_data['metadata'].keys()).union(oulucasia_data['metadata'].keys()):
    merged_metadata[key] = casia2_data['metadata'].get(key, 0) + oulucasia_data['metadata'].get(key, 0)

# Add the merged metadata to the merged data
merged_data['metadata'] = merged_metadata

# Save the merged data back to the file
with open('splits/new/merged_data_splits.json', 'w') as f:
    json.dump(merged_data, f)

### Preprocess

In [ ]:
import json
import pathlib
import os
import click
import cv2

from deepface import DeepFace


class FacePreprocessor:
    def __init__(
        self,
        train_split_pth,
        test_split_pth,
        val_split_pth,
        new_train_vl_pth,
        new_train_ni_pth,
        new_test_vl_pth,
        new_test_ni_pth,
        new_val_ni_pth,
        new_val_vl_pth,
        detector_backend,
        target_size,
        new_train_split_pth=None,
        new_test_split_pth=None,
        new_val_split_pth=None,
    ):
        self.train_split_pth = train_split_pth
        self.test_split_pth = test_split_pth
        self.val_split_pth = val_split_pth
        self.new_train_vl_pth = new_train_vl_pth
        self.new_train_ni_pth = new_train_ni_pth
        self.new_test_vl_pth = new_test_vl_pth
        self.new_test_ni_pth = new_test_ni_pth
        self.new_val_ni_pth = new_val_ni_pth
        self.new_val_vl_pth = new_val_vl_pth
        self.detector_backend = detector_backend
        self.target_size = target_size
        self.new_train_split = new_train_split_pth
        self.new_test_split = new_test_split_pth
        self.new_val_split = new_val_split_pth

    def detect_and_align_face(self, image_fp):
        try:
            face_objs = DeepFace.extract_faces(
                img_path=image_fp,
                target_size=self.target_size,
                detector_backend=self.detector_backend,
                enforce_detection=False,
            )
        except Exception as e:
            print(f"ERROR at {image_fp}", e)
            return None

        if len(face_objs) != 1:
            print("NOT FOUND OR MULTIPLE FACES!")
            return None

        face = face_objs[0]["face"]

        return face

    def preprocess_part(self, fps, target_fp, spectra):
        # prepare filepath
        os.makedirs(target_fp, exist_ok=True)

        # align faces for all images
        i = 0
        preprocessed_fps = []
        for fp in fps:
            new_filename = "-".join(pathlib.PurePath(fp).parts[-3:])
            target_path = os.path.join(target_fp, new_filename)

            aligned_face = self.detect_and_align_face(fp)

            if aligned_face is None:
                continue

            aligned_face = 255 * aligned_face[:, :, ::-1]

            cv2.imwrite(target_path, aligned_face)

            print(f"#{i} {spectra} Stored: {new_filename}")
            i += 1
            preprocessed_fps.append(target_path)

        return preprocessed_fps

    def preprocess_split(self, split_pth, new_vl_path, new_ni_pth):
        with open(split_pth, "r") as f:
            paths = json.load(f)

        vl_preproc_fps = self.preprocess_part(paths["vl"], new_vl_path, "vl")
        ni_preproc_fps = self.preprocess_part(paths["ni"], new_ni_pth, "ni")

        return {"vl": vl_preproc_fps, "ni": ni_preproc_fps}

    def preprocess(self):
#         preprocess train split
        train_fps = self.preprocess_split(
            self.train_split_pth, self.new_train_vl_pth, self.new_train_ni_pth
        )

        if self.new_train_split:
            with open(self.new_train_split, "w") as f:
                json.dump(train_fps, f)

        # preprocess test split
        test_fps = self.preprocess_split(
            self.test_split_pth, self.new_test_vl_pth, self.new_test_ni_pth
        )

        if self.new_test_split:
            with open(self.new_test_split, "w") as f:
                json.dump(test_fps, f)

        # preprocess val split
        val_fps = self.preprocess_split(
            self.val_split_pth, self.new_val_vl_pth, self.new_val_ni_pth
        )

        if self.new_val_split:
            with open(self.new_val_split, "w") as f:
                json.dump(val_fps, f)

In [ ]:
with open('splits/new/merged_data_splits.json') as f:
    merged_data = json.load(f)

In [ ]:
with open('_train_split_tmp.json', 'w') as f:
    json.dump({'ni': merged_data['nir_train'], 'vl': merged_data['vis_train']}, f)
with open('_test_split_tmp.json', 'w') as f:
    json.dump({'ni': merged_data['nir_test'], 'vl': merged_data['vis_test']}, f)
with open('_val_split_tmp.json', 'w') as f:
    json.dump({'ni': merged_data['nir_val'], 'vl': merged_data['vis_val']}, f)

preprocessor = FacePreprocessor(
    train_split_pth='_train_split_tmp.json',
    test_split_pth='_test_split_tmp.json',
    val_split_pth='_val_split_tmp.json',
    new_train_vl_pth='data/casia-oulucasia-unpaired/A/train',
    new_train_ni_pth='data/casia-oulucasia-unpaired/B/train',
    new_test_vl_pth='data/casia-oulucasia-unpaired/A/test',
    new_test_ni_pth='data/casia-oulucasia-unpaired/B/test',
    new_val_vl_pth='data/casia-oulucasia-unpaired/A/val',
    new_val_ni_pth='data/casia-oulucasia-unpaired/B/val',
    detector_backend='retinaface',
    target_size=(224, 224),
    new_train_split_pth='splits/new/preproc-merged-train',
    new_test_split_pth='splits/new/preproc-merged-test',
    new_val_split_pth='splits/new/preproc-merged-val',
)

preprocessor.preprocess()
               
os.remove('_train_split_tmp.json')
os.remove('_test_split_tmp.json')
os.remove('_val_split_tmp.json')

In [ ]:
merged_data.keys()

## Create-buaa-w_and_wo_stripes-train_test

In [ ]:
import os
import numpy as np
import random
from sklearn.model_selection import train_test_split
from PIL import Image
from deepface import DeepFace
from skeleton.inference import CenterFace
import cv2
import glob

__DATA_FOLDER = 'data/BUAA/BUAAVISNIR/'
__TARGET_SIZE = (224, 224)
__TARGET_FOLDER = 'data/buaa-w_and_wo_stripes-train_test/'

centerface = CenterFace()


def fill_black(image):
    height = image.shape[0]
    width = image.shape[1]
    max_dim = max([width, height])

    black_image = np.zeros((max_dim, max_dim, image.shape[-1]), dtype=np.uint8)

    width_padding = max_dim - width
    height_padding = max_dim - height
    y1 = int(height_padding/2)
    y2 = int(max_dim - height_padding/2)
    x1 = int(width_padding/2)
    x2 = int(max_dim - width_padding/2)

    black_image[y1:y2, x1:x2, :] = image

    return black_image


def operation_X(image_pth_A, image_pth_B):
    # load
    image_np_A = cv2.imread(str(image_pth_A))
    image_np_A = cv2.cvtColor(image_np_A, cv2.COLOR_BGR2RGB)
    image_np_B = cv2.imread(str(image_pth_B))
    image_np_B = cv2.cvtColor(image_np_B, cv2.COLOR_BGR2RGB)

    # expects 3 channels
    if image_np_A.shape[2] == 1:
        image_np_A = np.concatenate([image_np_A] * 3, axis=-1)

    # predict
    dets, lms = centerface(image_np_A, threshold=0.35)

    det = dets[0]
    width = det[2] - det[0]
    height = det[3] - det[1]
    bigger_dimension = max([width, height])
    width_padding = bigger_dimension - width
    height_padding = bigger_dimension - height

    face_A = image_np_A[int(det[1]-width_padding//2):int(det[3]+width_padding//2),
                        int(det[0]-width_padding//2):int(det[2]+width_padding//2), :]
    face_B = image_np_B[int(det[1]-width_padding//2):int(det[3]+width_padding//2),
                        int(det[0]-width_padding//2):int(det[2]+width_padding//2), :]

    face_A = cv2.resize(face_A, __TARGET_SIZE)
    face_B = cv2.resize(face_B, __TARGET_SIZE).astype(np.uint8)
    face_A_np = np.mean(face_A, axis=2)
    face_A = np.stack([face_A_np, face_A_np, face_A_np],
                      axis=2).astype(np.uint8)

    display(Image.fromarray(np.concatenate([face_A, face_B], axis=1)))

    return face_A, face_B


def operation_Y(image_pth_A, image_pth_B):
    face_objs_A = DeepFace.extract_faces(
        img_path=image_pth_A,
        target_size=__TARGET_SIZE,
        detector_backend="retinaface",
        enforce_detection=False,
        align=False
    )

    face_B_np = np.mean(cv2.imread(image_pth_B), axis=2)
    face_B_np = np.stack([face_B_np, face_B_np, face_B_np], axis=2)

    face_A = (face_objs_A[0]['face'] * 255).astype(np.uint8)
    ar = face_objs_A[0]['facial_area']
    face_B = face_B_np[ar['y']:ar['y']+ar['h'], ar['x']:ar['x']+ar['w']]

    face_B = fill_black(face_B)
    face_B = cv2.resize(face_B, __TARGET_SIZE).astype(np.uint8)

    face_A_np = np.mean(face_A, axis=2)
    face_A = np.stack([face_A_np, face_A_np, face_A_np],
                      axis=2).astype(np.uint8)

    display(Image.fromarray(np.concatenate([face_A, face_B], axis=1)))

    return face_A, face_B


# Get the list of patient ids
patient_ids = [name for name in os.listdir(
    __DATA_FOLDER) if os.path.isdir(os.path.join(__DATA_FOLDER, name))]

# Split the patient ids into train, validation, and test sets
train_ids, test_ids = train_test_split(
    patient_ids, test_size=0.2, random_state=42)
# train_ids, val_ids = train_test_split(train_ids, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2


# Function to process and save images
def process_and_save_images(ids, split):
    filepaths_A = []
    filepaths_B = []
    invalid = []
    for _id in sorted(ids):
        patient_folder = os.path.join(__DATA_FOLDER, _id)
        dir_content = [i for i in os.listdir(
            patient_folder) if i[-3:] == 'bmp']
        image_files = sorted(dir_content, key=lambda x: int(
            x.split('.')[0]))  # Sort the files to ensure pairs
        image_files = [i for i in image_files if int(i.split('.')[0]) <= 28]
        for file_A, file_B in zip(image_files[0::2], image_files[1::2]):
            print(str(os.path.join(patient_folder, file_A)),
                  str(os.path.join(patient_folder, file_B)))

            img_path_A = os.path.join(patient_folder, file_A)
            img_path_B = os.path.join(patient_folder, file_B)

            try:
                if random.choice([True, False]):
                    img_A, img_B = operation_X(img_path_A, img_path_B)
                else:
                    img_A, img_B = operation_Y(img_path_A, img_path_B)
            except:
                invalid.append(img_path_A)
                continue

            # Save the images
            Image.fromarray(img_A).save(
                os.path.join(split + 'A', f'{_id}_{file_A}'))
            Image.fromarray(img_B).save(
                os.path.join(split + 'B', f'{_id}_{file_B}'))

            filepaths_A.append(
                str(os.path.join(split + 'A', f'{_id}_{file_A}')))
            filepaths_B.append(
                str(os.path.join(split + 'B', f'{_id}_{file_B}')))

    return filepaths_A, filepaths_B, invalid

In [ ]:
# Process and save images for each split
out_train = process_and_save_images(train_ids, __TARGET_FOLDER + 'train')
# out_val = process_and_save_images(val_ids, __TARGET_FOLDER + 'val')
out_test = process_and_save_images(test_ids, __TARGET_FOLDER + 'test')

## AffectNet prep

In [ ]:
import json

In [ ]:
with open('splits/new/merged_data_splits.json', 'r') as f:
    casia_oulucasia_split = json.loads(f.read())

In [ ]:
# Load the existing data from the two files
with open('splits/new/casia2_data_splits.json', 'r') as f:
    casia2_data = json.load(f)
with open('splits/new/oulucasia_data_splits.json', 'r') as f:
    oulucasia_data = json.load(f)

In [ ]:
# Merge the data
merged_data = {}
for key in set(casia2_data.keys()).union(oulucasia_data.keys()):
    if key != 'metadata':
        merged_data[key] = casia2_data.get(key, []) + oulucasia_data.get(key, [])

# Merge the metadata
merged_metadata = {}
for key in set(casia2_data['metadata'].keys()).union(oulucasia_data['metadata'].keys()):
    merged_metadata[key] = casia2_data['metadata'].get(key, 0) + oulucasia_data['metadata'].get(key, 0)

# Add the merged metadata to the merged data
merged_data['metadata'] = merged_metadata

In [ ]:
merged_data['nir_rest']

In [ ]:
import json
import pathlib
import os
import click
import cv2

from deepface import DeepFace


def detect_and_align_face(image_fp):
    try:
        face_objs = DeepFace.extract_faces(
            img_path=image_fp,
            target_size=(256, 256),
            detector_backend="retinaface",
            enforce_detection=False,
        )
    except Exception as e:
        print(f"ERROR at {image_fp}", e)
        return None

    if len(face_objs) != 1:
        print("NOT FOUND OR MULTIPLE FACES!")
        return None

    face = face_objs[0]["face"]

    return face

def preprocess_part( fps, target_fp, spectra):
    # prepare filepath
    os.makedirs(target_fp, exist_ok=True)

    # align faces for all images
    i = 0
    preprocessed_fps = []
    for fp in fps:
        new_filename = "-".join(pathlib.PurePath(fp).parts[-3:])
        target_path = os.path.join(target_fp, new_filename)

        aligned_face = detect_and_align_face(fp)

        if aligned_face is None:
            continue

        aligned_face = 255 * aligned_face[:, :, ::-1]

        cv2.imwrite(target_path, aligned_face)

        print(f"#{i} {spectra} Stored: {new_filename}")
        i += 1
        preprocessed_fps.append(target_path)

    return preprocessed_fps

In [ ]:
preproc_casia_fps = preprocess_part(merged_data['nir_rest'], "data/for_unpaired-casia_preprocessed_rest_of_NIR/", "NIR")

In [ ]:
import os
__BUAA_PREPROC_FOLDER = 'data/Z_PreprocImg-BUAA-centerface-gray-averaged/BUAAVISNIR'

buaa_nir_images = []
for root, dirs, files in os.walk(__BUAA_PREPROC_FOLDER):
    for file in files:
        if int(file.split('.')[0]) % 2 == 0:
            buaa_nir_images.append(os.path.join(root, file))

In [ ]:
buaa_nir_images

In [ ]:
__AFFECTNET_PREPROC_TRAIN_FOLDER = 'data/AffectNet-8Labels/train_set/images'

retrieve_frist_n_images = len(buaa_nir_images) + len(merged_data['nir_rest'])
affectnet_vis_images = []
for i, f_nms in enumerate(sorted(os.listdir(__AFFECTNET_PREPROC_TRAIN_FOLDER), key=lambda x: int(x.split('.')[0]))):
    if i > retrieve_frist_n_images:
        break
    affectnet_vis_images.append(os.path.join(__AFFECTNET_PREPROC_TRAIN_FOLDER, f_nms))

In [ ]:
affectnet_vis_images

In [ ]:
import math
a = [0.,0.5]
print(a)

x = math.sqrt(a[0]*a[0] + a[1]*a[1])
print("radius", x, "-", "in" if x <=1 else "out")
r = 1 - x
print("rest to boundary", r)
enl = 1/(1 - a[0])
print("enlarge", enl)
print([a[0]*enl, a[1]*enl])

In [ ]:
os.mkdir('data/unpaired_additional_train-affectnet_vis_and_buaa_casia_nir', )
os.mkdir('data/unpaired_additional_train-affectnet_vis_and_buaa_casia_nir/vis', )
os.mkdir('data/unpaired_additional_train-affectnet_vis_and_buaa_casia_nir/nir', )

In [ ]:
import shutil

for i, (vis_pth, nir_pth) in enumerate(zip(affectnet_vis_images, buaa_nir_images + preproc_casia_fps)):
#     vis_new_fp = 'data/unpaired_additional_train-affectnet_vis_and_buaa_casia_nir/vis/' + "affnet-" + vis_pth.split('/')[-1]
    nir_new_fp = 'data/unpaired_additional_train-affectnet_vis_and_buaa_casia_nir/nir/' + "buaa-casia-" + str(i) + "." + (nir_pth.split('/')[-1]).split('.')[-1]
    
#     shutil.copyfile(vis_pth, vis_new_fp)
    shutil.copyfile(nir_pth, nir_new_fp)

In [ ]:
merged_data['nir_rest']

## Playground

In [2]:
from skeleton.inference import Inference, CenterFace
import cv2

models = {
    "face_detector": {
        "net_type": Inference.net_type.FACE_DETECTOR_RETINAFACE,
        "remove_black_stripes": True,
        "display_images": True,
        "save_image_to_folder": "Custom_DB/custom_nir_wo_stripes",
    },
    "spectrum_translator": {
        "net_type": None, 
        "pth_to_onnx": 'models/cyclegan-snellius-casia-oulucasia-double_gen_op/NIR2VIS_cyclegan_snellius_casia_oulucasia_double_gen_opt-GA-20epoch.onnx',
        "input_as_avg_grayscale": False,
        "output_as_avg_grayscale": True,
    },
    "fer": {
        "net_type": None,
        "pth_to_onnx": "models/mobilenet_NIR/mobilenet_on_AffectNet-NIR/mobilenet_aff_nir-aff_continue.onnx",
        "va_to_circumplex_model": False,
    }
}
inf = Inference(models, None, verbose=True, debug=True)

2024-02-22 12:24:27.039128: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-22 12:24:27.093856: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-22 12:24:27.095327: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-22 12:24:28.002829: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Using '{'net_type': <net_type.FACE_DETECTOR_RETINAFACE: 'G'>, 'remove_black_stripes': True, 'display_images': True, 'save_image_to_folder': 'Custom_DB/custom_nir_wo_stripes'}' as face detector model


In [ ]:
out = inf.infer_from_folder("Custom_DB/custom_nir/")

In [74]:
import pandas as pd

df_all = pd.read_csv("df_cyclegan_all.csv")

In [75]:
df_chosen = df_all[
    (df_all["expression"].notna()) &
#     (df_all["db"] != "oulucasia") &
    (df_all["db"] != "affectnet") &
    (df_all["spectra"] == "nir") &
    (df_all["buaa-image_type"]!= 'tilt')
      ]

In [76]:
df_chosen.to_csv('df_fer_customdb_buaa_oulucasia2.csv')

In [35]:
import pandas as pd
import shutil
import os

destination_folder = "fer_custom_db_with_buaa_and_oulucasia"

# Ensure the destination folder exists
os.makedirs(destination_folder, exist_ok=True)

for index, row in df_chosen.iterrows():
    source_file_path = row['filepath']
    destination_file_name = row['adjusted_unique_filename']
    destination_file_path = os.path.join(destination_folder, destination_file_name)
    
    # Copy the file
    shutil.copy(source_file_path, destination_file_path)

In [41]:
from skeleton.inference import Inference

import cv2

models = {
    "face_detector": {
        "net_type": Inference.net_type.FACE_DETECTOR_RETINAFACE,
        "remove_black_stripes": False,
        "display_images": True,
        "save_image_to_folder": "fer_custom_db_with_buaa_and_oulucasia_detected_wo_stripes",
    },
    "spectrum_translator": {
        "net_type": None, 
        "pth_to_onnx": 'models/cyclegan-snellius-casia-oulucasia-double_gen_op/NIR2VIS_cyclegan_snellius_casia_oulucasia_double_gen_opt-GA-20epoch.onnx',
        "input_as_avg_grayscale": False,
        "output_as_avg_grayscale": True,
    },
    "fer": {
        "net_type": None,
        "pth_to_onnx": "models/mobilenet_NIR/mobilenet_on_AffectNet-NIR/mobilenet_aff_nir-aff_continue.onnx",
        "va_to_circumplex_model": False,
    }
}
inf = Inference(models, None, verbose=True, debug=True)

Using '{'net_type': <net_type.FACE_DETECTOR_RETINAFACE: 'G'>, 'remove_black_stripes': False, 'display_images': True, 'save_image_to_folder': 'fer_custom_db_with_buaa_and_oulucasia_detected_wo_stripes'}' as face detector model


In [ ]:
out = inf.infer_instant_from_folder("fer_custom_db_with_buaa_and_oulucasia")